In [ ]:
## This file implements neural networks and logistic regression on p0006presabsSTCC_quant.
## Since around 68% of the original data is classified as 0, we first construct fully-connected neural networks on orginal
## splited data, which leads to the accuracy of 75.32% after improvement.
## We then implement the over-sampling method and the combination of over- and under-sampling method.
## For fully-connected neural networks, the accuracy is 75% for combination data, and 65.38% for over-sampling data after improvements.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## However, with the new datasets, the accuracy becomes 65.91% for combination data and 82.31% for over-sampling data.
## For logistic regression, the accuracies are 88.64% for combination data and 76.92% for over-sampling data.
## For random forest, the accuracy is 97.73% for combination and 81.73% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 93.99% for combination data, and 73.95% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0006presabsSTCC_quant.csv')
df.shape

(255, 88)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.537000
1      0.325833
2      0.630333
3      0.422500
4      0.433833
5      0.505833
6      0.431167
7      0.228333
8      0.547167
9      0.436667
10     0.366333
11     0.903500
12     0.431667
13     0.327500
14     0.339833
15     0.217000
16     0.441333
17     0.442667
18     0.794833
19     0.215000
20     0.426167
21     0.611667
22     0.322833
23     0.219667
24     1.004333
25     0.175000
26     0.837667
27     0.184667
28     0.195833
29     0.711667
         ...   
225    0.546400
226    0.545700
227    0.606100
228    0.964600
229    0.775600
230    0.441500
231    0.356167
232    0.437167
233    0.698667
234    0.455333
235    0.914500
236    0.231167
237    0.213667
238    0.341833
239    0.361667
240    0.342833
241    0.650667
242    0.287667
243    0.301167
244    0.514333
245    0.232500
246    0.429833
247    0.335667
248    0.191500
249    0.327667
250    0.449167
251    0.306167
252    0.388500
253    0.374833
254    0.330833
Name: pheno, Length: 255

In [4]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno,ST,CC
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.537000,5,5
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.325833,8,8
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.630333,5,5
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.422500,5,5
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.433833,5,5


In [5]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [6]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [7]:
df['pheno']
df['pheno'].value_counts()

0    173
1     82
Name: pheno, dtype: int64

In [8]:
df.shape

(255, 88)

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 87)

In [11]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno,ST,CC
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 86) (255,)


In [75]:
# split into train, validation, and test data (original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [76]:
############# Fully-Connected Neural Network ################

In [77]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [78]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [79]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [80]:
model.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 555us/step - loss: 9.2790 - accuracy: 0.5112 - val_loss: 0.8010 - val_accuracy: 0.4805
Epoch 2/1000
178/178 [==============================] - 0s 88us/step - loss: 0.7406 - accuracy: 0.5562 - val_loss: 0.7326 - val_accuracy: 0.4026
Epoch 3/1000
178/178 [==============================] - 0s 178us/step - loss: 0.7028 - accuracy: 0.5056 - val_loss: 0.7012 - val_accuracy: 0.5584
Epoch 4/1000
178/178 [==============================] - 0s 170us/step - loss: 0.6806 - accuracy: 0.6124 - val_loss: 0.6855 - val_accuracy: 0.5844
Epoch 5/1000
178/178 [==============================] - 0s 148us/step - loss: 0.6722 - accuracy: 0.6348 - val_loss: 0.6821 - val_accuracy: 0.6494
Epoch 6/1000
178/178 [==============================] - 0s 201us/step - loss: 0.6666 - accuracy: 0.6685 - val_loss: 0.6729 - val_accuracy: 0.6494
Epoch 7/1000
178/178 [==============================] - 0s 125us/step - loss: 0.

178/178 [==============================] - 0s 168us/step - loss: 0.6089 - accuracy: 0.6629 - val_loss: 0.6204 - val_accuracy: 0.7013
Epoch 57/1000
178/178 [==============================] - 0s 135us/step - loss: 0.5985 - accuracy: 0.6798 - val_loss: 0.6192 - val_accuracy: 0.6753
Epoch 58/1000
178/178 [==============================] - 0s 117us/step - loss: 0.6005 - accuracy: 0.6742 - val_loss: 0.6187 - val_accuracy: 0.6753
Epoch 59/1000
178/178 [==============================] - 0s 107us/step - loss: 0.5994 - accuracy: 0.6798 - val_loss: 0.6194 - val_accuracy: 0.6753
Epoch 60/1000
178/178 [==============================] - 0s 103us/step - loss: 0.6009 - accuracy: 0.6798 - val_loss: 0.6194 - val_accuracy: 0.7013
Epoch 61/1000
178/178 [==============================] - 0s 99us/step - loss: 0.5979 - accuracy: 0.6742 - val_loss: 0.6188 - val_accuracy: 0.6753
Epoch 62/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5943 - accuracy: 0.6798 - val_loss: 0.6188 - val_acc

Epoch 112/1000
178/178 [==============================] - 0s 117us/step - loss: 0.5829 - accuracy: 0.6798 - val_loss: 0.6137 - val_accuracy: 0.7013
Epoch 113/1000
178/178 [==============================] - 0s 107us/step - loss: 0.5799 - accuracy: 0.6910 - val_loss: 0.6085 - val_accuracy: 0.6753
Epoch 114/1000
178/178 [==============================] - 0s 89us/step - loss: 0.5967 - accuracy: 0.6629 - val_loss: 0.6069 - val_accuracy: 0.7273
Epoch 115/1000
178/178 [==============================] - ETA: 0s - loss: 0.5809 - accuracy: 0.75 - 0s 92us/step - loss: 0.5874 - accuracy: 0.6854 - val_loss: 0.6073 - val_accuracy: 0.7143
Epoch 116/1000
178/178 [==============================] - 0s 98us/step - loss: 0.5830 - accuracy: 0.6910 - val_loss: 0.6110 - val_accuracy: 0.6753
Epoch 117/1000
178/178 [==============================] - 0s 109us/step - loss: 0.5840 - accuracy: 0.6742 - val_loss: 0.6112 - val_accuracy: 0.6753
Epoch 118/1000
178/178 [==============================] - 0s 107us/step -

178/178 [==============================] - 0s 112us/step - loss: 0.5762 - accuracy: 0.6966 - val_loss: 0.5951 - val_accuracy: 0.7273
Epoch 168/1000
178/178 [==============================] - 0s 126us/step - loss: 0.5680 - accuracy: 0.7247 - val_loss: 0.6051 - val_accuracy: 0.6753
Epoch 169/1000
178/178 [==============================] - 0s 109us/step - loss: 0.5750 - accuracy: 0.7022 - val_loss: 0.5949 - val_accuracy: 0.7273
Epoch 170/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5861 - accuracy: 0.6910 - val_loss: 0.6075 - val_accuracy: 0.6753
Epoch 171/1000
178/178 [==============================] - ETA: 0s - loss: 0.5497 - accuracy: 0.78 - 0s 152us/step - loss: 0.5649 - accuracy: 0.7135 - val_loss: 0.6112 - val_accuracy: 0.6753
Epoch 172/1000
178/178 [==============================] - 0s 131us/step - loss: 0.5739 - accuracy: 0.7022 - val_loss: 0.6036 - val_accuracy: 0.6753
Epoch 173/1000
178/178 [==============================] - 0s 127us/step - loss: 0.571

178/178 [==============================] - 0s 119us/step - loss: 0.5618 - accuracy: 0.7135 - val_loss: 0.6290 - val_accuracy: 0.6364
Epoch 223/1000
178/178 [==============================] - 0s 117us/step - loss: 0.5733 - accuracy: 0.6966 - val_loss: 0.6001 - val_accuracy: 0.6753
Epoch 224/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5686 - accuracy: 0.6854 - val_loss: 0.5861 - val_accuracy: 0.7403
Epoch 225/1000
178/178 [==============================] - 0s 115us/step - loss: 0.5673 - accuracy: 0.7022 - val_loss: 0.5860 - val_accuracy: 0.7403
Epoch 226/1000
178/178 [==============================] - 0s 118us/step - loss: 0.5549 - accuracy: 0.7191 - val_loss: 0.6192 - val_accuracy: 0.6364
Epoch 227/1000
178/178 [==============================] - 0s 132us/step - loss: 0.5866 - accuracy: 0.6910 - val_loss: 0.5855 - val_accuracy: 0.7403
Epoch 228/1000
178/178 [==============================] - 0s 131us/step - loss: 0.5590 - accuracy: 0.7191 - val_loss: 0.5861 - 

...

178/178 [==============================] - 0s 104us/step - loss: 0.5164 - accuracy: 0.7360 - val_loss: 0.5766 - val_accuracy: 0.7273
Epoch 773/1000
178/178 [==============================] - 0s 116us/step - loss: 0.5148 - accuracy: 0.7360 - val_loss: 0.5810 - val_accuracy: 0.7273
Epoch 774/1000
178/178 [==============================] - 0s 113us/step - loss: 0.5275 - accuracy: 0.7360 - val_loss: 0.5748 - val_accuracy: 0.7273
Epoch 775/1000
178/178 [==============================] - 0s 152us/step - loss: 0.5092 - accuracy: 0.7416 - val_loss: 0.5740 - val_accuracy: 0.7273
Epoch 776/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5125 - accuracy: 0.7360 - val_loss: 0.5899 - val_accuracy: 0.7013
Epoch 777/1000
178/178 [==============================] - 0s 123us/step - loss: 0.5160 - accuracy: 0.7303 - val_loss: 0.5801 - val_accuracy: 0.7403
Epoch 778/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5093 - accuracy: 0.7416 - val_loss: 0.6207 - 

178/178 [==============================] - 0s 125us/step - loss: 0.5278 - accuracy: 0.7303 - val_loss: 0.5777 - val_accuracy: 0.7273
Epoch 828/1000
178/178 [==============================] - 0s 121us/step - loss: 0.5167 - accuracy: 0.7303 - val_loss: 0.5731 - val_accuracy: 0.7273
Epoch 829/1000
178/178 [==============================] - 0s 131us/step - loss: 0.5104 - accuracy: 0.7360 - val_loss: 0.5737 - val_accuracy: 0.7273
Epoch 830/1000
178/178 [==============================] - 0s 130us/step - loss: 0.5208 - accuracy: 0.7360 - val_loss: 0.5699 - val_accuracy: 0.7273
Epoch 831/1000
178/178 [==============================] - 0s 135us/step - loss: 0.5077 - accuracy: 0.7416 - val_loss: 0.5831 - val_accuracy: 0.7403
Epoch 832/1000
178/178 [==============================] - 0s 135us/step - loss: 0.5144 - accuracy: 0.7360 - val_loss: 0.6307 - val_accuracy: 0.6234
Epoch 833/1000
178/178 [==============================] - 0s 109us/step - loss: 0.5254 - accuracy: 0.7247 - val_loss: 0.5701 - 

178/178 [==============================] - 0s 129us/step - loss: 0.5457 - accuracy: 0.7191 - val_loss: 0.5792 - val_accuracy: 0.7273
Epoch 883/1000
178/178 [==============================] - 0s 134us/step - loss: 0.5141 - accuracy: 0.7360 - val_loss: 0.5706 - val_accuracy: 0.7273
Epoch 884/1000
178/178 [==============================] - 0s 116us/step - loss: 0.5049 - accuracy: 0.7360 - val_loss: 0.6100 - val_accuracy: 0.6753
Epoch 885/1000
178/178 [==============================] - 0s 91us/step - loss: 0.5058 - accuracy: 0.7416 - val_loss: 0.5743 - val_accuracy: 0.7273
Epoch 886/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5080 - accuracy: 0.7303 - val_loss: 0.5688 - val_accuracy: 0.7273
Epoch 887/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5255 - accuracy: 0.7191 - val_loss: 0.5937 - val_accuracy: 0.7143
Epoch 888/1000
178/178 [==============================] - 0s 90us/step - loss: 0.5177 - accuracy: 0.7191 - val_loss: 0.5699 - val

178/178 [==============================] - 0s 126us/step - loss: 0.5068 - accuracy: 0.7360 - val_loss: 0.5737 - val_accuracy: 0.7143
Epoch 938/1000
178/178 [==============================] - 0s 110us/step - loss: 0.5028 - accuracy: 0.7416 - val_loss: 0.5667 - val_accuracy: 0.7143
Epoch 939/1000
178/178 [==============================] - 0s 145us/step - loss: 0.5115 - accuracy: 0.7303 - val_loss: 0.5898 - val_accuracy: 0.6883
Epoch 940/1000
178/178 [==============================] - 0s 129us/step - loss: 0.5152 - accuracy: 0.7303 - val_loss: 0.5770 - val_accuracy: 0.7273
Epoch 941/1000
178/178 [==============================] - 0s 127us/step - loss: 0.5073 - accuracy: 0.7416 - val_loss: 0.5787 - val_accuracy: 0.7013
Epoch 942/1000
178/178 [==============================] - 0s 118us/step - loss: 0.5059 - accuracy: 0.7191 - val_loss: 0.5801 - val_accuracy: 0.7273
Epoch 943/1000
178/178 [==============================] - 0s 129us/step - loss: 0.5117 - accuracy: 0.7360 - val_loss: 0.5730 - 

178/178 [==============================] - 0s 97us/step - loss: 0.4976 - accuracy: 0.7416 - val_loss: 0.6015 - val_accuracy: 0.6753
Epoch 993/1000
178/178 [==============================] - 0s 109us/step - loss: 0.5064 - accuracy: 0.7360 - val_loss: 0.5687 - val_accuracy: 0.7273
Epoch 994/1000
178/178 [==============================] - 0s 114us/step - loss: 0.5002 - accuracy: 0.7472 - val_loss: 0.5644 - val_accuracy: 0.7403
Epoch 995/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4987 - accuracy: 0.7640 - val_loss: 0.5607 - val_accuracy: 0.7532
Epoch 996/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5071 - accuracy: 0.7472 - val_loss: 0.5705 - val_accuracy: 0.7143
Epoch 997/1000
178/178 [==============================] - 0s 130us/step - loss: 0.5103 - accuracy: 0.7191 - val_loss: 0.5696 - val_accuracy: 0.7143
Epoch 998/1000
178/178 [==============================] - 0s 114us/step - loss: 0.5083 - accuracy: 0.7247 - val_loss: 0.5605 - v

In [81]:
acc_test = model.evaluate(X_test, y_test)[1]
print('combination test accuracy: %.2f%%' % (acc_test*100))

77/77 [==============================] - 0s 111us/step
combination test accuracy: 72.73%


In [82]:
#### improve model
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [83]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [84]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 645us/step - loss: 16.6971 - accuracy: 0.5056 - val_loss: 0.6962 - val_accuracy: 0.6364
Epoch 2/1000
178/178 [==============================] - 0s 115us/step - loss: 1.0474 - accuracy: 0.5730 - val_loss: 0.6701 - val_accuracy: 0.6753
Epoch 3/1000
178/178 [==============================] - 0s 143us/step - loss: 0.6860 - accuracy: 0.6573 - val_loss: 0.6530 - val_accuracy: 0.6883
Epoch 4/1000
178/178 [==============================] - 0s 144us/step - loss: 0.6601 - accuracy: 0.6629 - val_loss: 0.6556 - val_accuracy: 0.6753
Epoch 5/1000
178/178 [==============================] - 0s 155us/step - loss: 0.6624 - accuracy: 0.6629 - val_loss: 0.6460 - val_accuracy: 0.6753
Epoch 6/1000
178/178 [==============================] - 0s 127us/step - loss: 0.6448 - accuracy: 0.6629 - val_loss: 0.6418 - val_accuracy: 0.6753
Epoch 7/1000
178/178 [==============================] - 0s 164us/step - loss: 

178/178 [==============================] - 0s 121us/step - loss: 0.6067 - accuracy: 0.6798 - val_loss: 0.6178 - val_accuracy: 0.6753
Epoch 57/1000
178/178 [==============================] - 0s 110us/step - loss: 0.6064 - accuracy: 0.6798 - val_loss: 0.6174 - val_accuracy: 0.6753
Epoch 58/1000
178/178 [==============================] - 0s 129us/step - loss: 0.6057 - accuracy: 0.6798 - val_loss: 0.6179 - val_accuracy: 0.6753
Epoch 59/1000
178/178 [==============================] - 0s 112us/step - loss: 0.6068 - accuracy: 0.6798 - val_loss: 0.6172 - val_accuracy: 0.6753
Epoch 60/1000
178/178 [==============================] - 0s 123us/step - loss: 0.6050 - accuracy: 0.6798 - val_loss: 0.6170 - val_accuracy: 0.6753
Epoch 61/1000
178/178 [==============================] - 0s 117us/step - loss: 0.6050 - accuracy: 0.6798 - val_loss: 0.6168 - val_accuracy: 0.6753
Epoch 62/1000
178/178 [==============================] - 0s 125us/step - loss: 0.6055 - accuracy: 0.6798 - val_loss: 0.6166 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 74us/step - loss: 0.5960 - accuracy: 0.6798 - val_loss: 0.6095 - val_accuracy: 0.6753
Epoch 113/1000
178/178 [==============================] - 0s 90us/step - loss: 0.5962 - accuracy: 0.6798 - val_loss: 0.6094 - val_accuracy: 0.6753
Epoch 114/1000
178/178 [==============================] - 0s 115us/step - loss: 0.5960 - accuracy: 0.6798 - val_loss: 0.6097 - val_accuracy: 0.6753
Epoch 115/1000
178/178 [==============================] - 0s 100us/step - loss: 0.5958 - accuracy: 0.6798 - val_loss: 0.6091 - val_accuracy: 0.6753
Epoch 116/1000
178/178 [==============================] - 0s 98us/step - loss: 0.5952 - accuracy: 0.6798 - val_loss: 0.6081 - val_accuracy: 0.6753
Epoch 117/1000
178/178 [==============================] - 0s 103us/step - loss: 0.5952 - accuracy: 0.6798 - val_loss: 0.6072 - val_accuracy: 0.6753
Epoch 118/1000
178/178 [==============================] - 0s 108us/step - loss: 0.5977 - accuracy: 0.6629 - val_los

178/178 [==============================] - 0s 98us/step - loss: 0.5893 - accuracy: 0.6798 - val_loss: 0.6016 - val_accuracy: 0.6753
Epoch 168/1000
178/178 [==============================] - 0s 110us/step - loss: 0.5916 - accuracy: 0.6685 - val_loss: 0.6011 - val_accuracy: 0.6753
Epoch 169/1000
178/178 [==============================] - 0s 101us/step - loss: 0.5897 - accuracy: 0.6798 - val_loss: 0.6017 - val_accuracy: 0.6753
Epoch 170/1000
178/178 [==============================] - 0s 128us/step - loss: 0.5878 - accuracy: 0.6798 - val_loss: 0.6004 - val_accuracy: 0.6753
Epoch 171/1000
178/178 [==============================] - 0s 118us/step - loss: 0.5904 - accuracy: 0.6685 - val_loss: 0.6023 - val_accuracy: 0.6753
Epoch 172/1000
178/178 [==============================] - 0s 141us/step - loss: 0.5877 - accuracy: 0.6798 - val_loss: 0.6009 - val_accuracy: 0.6753
Epoch 173/1000
178/178 [==============================] - 0s 124us/step - loss: 0.5876 - accuracy: 0.6798 - val_loss: 0.6004 - v

178/178 [==============================] - 0s 122us/step - loss: 0.5814 - accuracy: 0.6798 - val_loss: 0.5940 - val_accuracy: 0.6883
Epoch 223/1000
178/178 [==============================] - 0s 125us/step - loss: 0.5840 - accuracy: 0.6910 - val_loss: 0.5936 - val_accuracy: 0.6883
Epoch 224/1000
178/178 [==============================] - 0s 121us/step - loss: 0.5794 - accuracy: 0.6798 - val_loss: 0.5937 - val_accuracy: 0.6883
Epoch 225/1000
178/178 [==============================] - 0s 114us/step - loss: 0.5870 - accuracy: 0.6742 - val_loss: 0.5943 - val_accuracy: 0.7143
Epoch 226/1000
178/178 [==============================] - 0s 126us/step - loss: 0.5793 - accuracy: 0.6854 - val_loss: 0.5938 - val_accuracy: 0.6753
Epoch 227/1000
178/178 [==============================] - 0s 112us/step - loss: 0.5813 - accuracy: 0.6798 - val_loss: 0.5951 - val_accuracy: 0.6753
Epoch 228/1000
178/178 [==============================] - 0s 115us/step - loss: 0.5837 - accuracy: 0.6798 - val_loss: 0.5936 - 

...

178/178 [==============================] - 0s 99us/step - loss: 0.5274 - accuracy: 0.7303 - val_loss: 0.5410 - val_accuracy: 0.7273
Epoch 773/1000
178/178 [==============================] - 0s 88us/step - loss: 0.5205 - accuracy: 0.7303 - val_loss: 0.5361 - val_accuracy: 0.7403
Epoch 774/1000
178/178 [==============================] - 0s 118us/step - loss: 0.5032 - accuracy: 0.7416 - val_loss: 0.5965 - val_accuracy: 0.6364
Epoch 775/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5033 - accuracy: 0.7360 - val_loss: 0.5334 - val_accuracy: 0.7403
Epoch 776/1000
178/178 [==============================] - 0s 111us/step - loss: 0.5024 - accuracy: 0.7416 - val_loss: 0.5289 - val_accuracy: 0.7273
Epoch 777/1000
178/178 [==============================] - 0s 95us/step - loss: 0.5061 - accuracy: 0.7416 - val_loss: 0.6194 - val_accuracy: 0.6364
Epoch 778/1000
178/178 [==============================] - 0s 114us/step - loss: 0.5988 - accuracy: 0.6798 - val_loss: 0.5526 - val

178/178 [==============================] - 0s 105us/step - loss: 0.5044 - accuracy: 0.7360 - val_loss: 0.5354 - val_accuracy: 0.7273
Epoch 828/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4984 - accuracy: 0.7416 - val_loss: 0.5302 - val_accuracy: 0.7532
Epoch 829/1000
178/178 [==============================] - 0s 110us/step - loss: 0.4907 - accuracy: 0.7416 - val_loss: 0.5309 - val_accuracy: 0.7273
Epoch 830/1000
178/178 [==============================] - 0s 113us/step - loss: 0.4959 - accuracy: 0.7472 - val_loss: 0.5364 - val_accuracy: 0.7273
Epoch 831/1000
178/178 [==============================] - 0s 106us/step - loss: 0.4963 - accuracy: 0.7416 - val_loss: 0.5271 - val_accuracy: 0.7403
Epoch 832/1000
178/178 [==============================] - 0s 100us/step - loss: 0.5029 - accuracy: 0.7360 - val_loss: 0.5253 - val_accuracy: 0.7532
Epoch 833/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5534 - accuracy: 0.6966 - val_loss: 0.5858 - v

178/178 [==============================] - 0s 93us/step - loss: 0.4978 - accuracy: 0.7472 - val_loss: 0.5325 - val_accuracy: 0.7273
Epoch 883/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4955 - accuracy: 0.7472 - val_loss: 0.5339 - val_accuracy: 0.7273
Epoch 884/1000
178/178 [==============================] - 0s 119us/step - loss: 0.4847 - accuracy: 0.7416 - val_loss: 0.5391 - val_accuracy: 0.7403
Epoch 885/1000
178/178 [==============================] - 0s 105us/step - loss: 0.4873 - accuracy: 0.7472 - val_loss: 0.5563 - val_accuracy: 0.7143
Epoch 886/1000
178/178 [==============================] - 0s 98us/step - loss: 0.5254 - accuracy: 0.7247 - val_loss: 0.5322 - val_accuracy: 0.7273
Epoch 887/1000
178/178 [==============================] - 0s 109us/step - loss: 0.4922 - accuracy: 0.7416 - val_loss: 0.5396 - val_accuracy: 0.7273
Epoch 888/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4942 - accuracy: 0.7416 - val_loss: 0.5313 - va

178/178 [==============================] - 0s 96us/step - loss: 0.4875 - accuracy: 0.7416 - val_loss: 0.5344 - val_accuracy: 0.7273
Epoch 938/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4866 - accuracy: 0.7416 - val_loss: 0.5394 - val_accuracy: 0.7273
Epoch 939/1000
178/178 [==============================] - 0s 110us/step - loss: 0.4909 - accuracy: 0.7472 - val_loss: 0.5323 - val_accuracy: 0.7273
Epoch 940/1000
178/178 [==============================] - 0s 103us/step - loss: 0.4927 - accuracy: 0.7472 - val_loss: 0.5434 - val_accuracy: 0.7273
Epoch 941/1000
178/178 [==============================] - 0s 108us/step - loss: 0.4882 - accuracy: 0.7416 - val_loss: 0.5301 - val_accuracy: 0.7403
Epoch 942/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4963 - accuracy: 0.7303 - val_loss: 0.5501 - val_accuracy: 0.7013
Epoch 943/1000
178/178 [==============================] - 0s 108us/step - loss: 0.4921 - accuracy: 0.7360 - val_loss: 0.5349 - v

178/178 [==============================] - 0s 92us/step - loss: 0.4833 - accuracy: 0.7472 - val_loss: 0.5330 - val_accuracy: 0.7403
Epoch 993/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4872 - accuracy: 0.7360 - val_loss: 0.5345 - val_accuracy: 0.7403
Epoch 994/1000
178/178 [==============================] - 0s 118us/step - loss: 0.4945 - accuracy: 0.7360 - val_loss: 0.5358 - val_accuracy: 0.7273
Epoch 995/1000
178/178 [==============================] - 0s 114us/step - loss: 0.4836 - accuracy: 0.7416 - val_loss: 0.5362 - val_accuracy: 0.7273
Epoch 996/1000
178/178 [==============================] - 0s 106us/step - loss: 0.4881 - accuracy: 0.7416 - val_loss: 0.5397 - val_accuracy: 0.7532
Epoch 997/1000
178/178 [==============================] - 0s 101us/step - loss: 0.4836 - accuracy: 0.7416 - val_loss: 0.5287 - val_accuracy: 0.7403
Epoch 998/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4835 - accuracy: 0.7360 - val_loss: 0.5319 - val

In [85]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('combination test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 106us/step
combination test accuracy: 75.32%


In [13]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 77), (1, 68)]


In [14]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 173), (1, 173)]


In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 101 samples, validate on 44 samples
Epoch 1/1000
101/101 [==============================] - 0s 2ms/step - loss: 60.7662 - accuracy: 0.4653 - val_loss: 26.6723 - val_accuracy: 0.3409
Epoch 2/1000
101/101 [==============================] - 0s 135us/step - loss: 5.4290 - accuracy: 0.5347 - val_loss: 0.8029 - val_accuracy: 0.3864
Epoch 3/1000
101/101 [==============================] - 0s 193us/step - loss: 0.6691 - accuracy: 0.4851 - val_loss: 0.7610 - val_accuracy: 0.4091
Epoch 4/1000
101/101 [==============================] - 0s 243us/step - loss: 0.6665 - accuracy: 0.4851 - val_loss: 0.7783 - val_accuracy: 0.3864
Epoch 5/1000
101/101 [==============================] - 0s 201us/step - loss: 0.6638 - accuracy: 0.4851 - val_loss: 0.7705 - val_accuracy: 0.3864
Epoch 6/1000
101/101 [==============================] - 0s 204us/step - loss: 0.6611 - accuracy: 0.4851 - val_loss: 0.7616 - val_accuracy: 0.4091
Epoch 7/1000
101/101 [==============================] - 0s 196us/step - loss: 0

101/101 [==============================] - 0s 178us/step - loss: 0.6077 - accuracy: 0.7228 - val_loss: 0.7300 - val_accuracy: 0.5682
Epoch 57/1000
101/101 [==============================] - 0s 162us/step - loss: 0.6024 - accuracy: 0.7624 - val_loss: 0.7172 - val_accuracy: 0.5682
Epoch 58/1000
101/101 [==============================] - 0s 125us/step - loss: 0.6007 - accuracy: 0.7228 - val_loss: 0.7293 - val_accuracy: 0.5682
Epoch 59/1000
101/101 [==============================] - 0s 172us/step - loss: 0.5999 - accuracy: 0.7228 - val_loss: 0.7680 - val_accuracy: 0.5682
Epoch 60/1000
101/101 [==============================] - 0s 155us/step - loss: 0.6041 - accuracy: 0.7624 - val_loss: 0.7626 - val_accuracy: 0.5682
Epoch 61/1000
101/101 [==============================] - 0s 135us/step - loss: 0.6024 - accuracy: 0.7624 - val_loss: 0.7190 - val_accuracy: 0.5682
Epoch 62/1000
101/101 [==============================] - 0s 159us/step - loss: 0.5965 - accuracy: 0.7228 - val_loss: 0.7328 - val_ac

Epoch 112/1000
101/101 [==============================] - 0s 143us/step - loss: 0.5548 - accuracy: 0.7228 - val_loss: 0.7252 - val_accuracy: 0.5455
Epoch 113/1000
101/101 [==============================] - ETA: 0s - loss: 0.4896 - accuracy: 0.75 - 0s 171us/step - loss: 0.7049 - accuracy: 0.7525 - val_loss: 0.6578 - val_accuracy: 0.5227
Epoch 114/1000
101/101 [==============================] - 0s 142us/step - loss: 0.6055 - accuracy: 0.6436 - val_loss: 0.6720 - val_accuracy: 0.5455
Epoch 115/1000
101/101 [==============================] - 0s 128us/step - loss: 0.5674 - accuracy: 0.7228 - val_loss: 0.6918 - val_accuracy: 0.5455
Epoch 116/1000
101/101 [==============================] - 0s 128us/step - loss: 0.5635 - accuracy: 0.7228 - val_loss: 0.7392 - val_accuracy: 0.5455
Epoch 117/1000
101/101 [==============================] - 0s 137us/step - loss: 0.5623 - accuracy: 0.7228 - val_loss: 0.7883 - val_accuracy: 0.5682
Epoch 118/1000
101/101 [==============================] - 0s 167us/ste

101/101 [==============================] - 0s 117us/step - loss: 0.6428 - accuracy: 0.5743 - val_loss: 0.6488 - val_accuracy: 0.5227
Epoch 168/1000
101/101 [==============================] - 0s 105us/step - loss: 0.6407 - accuracy: 0.5743 - val_loss: 0.6526 - val_accuracy: 0.5227
Epoch 169/1000
101/101 [==============================] - 0s 143us/step - loss: 0.6590 - accuracy: 0.5743 - val_loss: 0.6533 - val_accuracy: 0.5227
Epoch 170/1000
101/101 [==============================] - 0s 134us/step - loss: 0.6419 - accuracy: 0.5743 - val_loss: 0.6524 - val_accuracy: 0.5227
Epoch 171/1000
101/101 [==============================] - 0s 125us/step - loss: 0.6404 - accuracy: 0.5743 - val_loss: 0.6507 - val_accuracy: 0.5227
Epoch 172/1000
101/101 [==============================] - 0s 131us/step - loss: 0.6395 - accuracy: 0.5743 - val_loss: 0.6473 - val_accuracy: 0.5227
Epoch 173/1000
101/101 [==============================] - 0s 163us/step - loss: 0.6390 - accuracy: 0.5743 - val_loss: 0.6441 - 

101/101 [==============================] - 0s 98us/step - loss: 0.6118 - accuracy: 0.5743 - val_loss: 0.6221 - val_accuracy: 0.5455
Epoch 223/1000
101/101 [==============================] - 0s 131us/step - loss: 0.6110 - accuracy: 0.5743 - val_loss: 0.6175 - val_accuracy: 0.5455
Epoch 224/1000
101/101 [==============================] - 0s 147us/step - loss: 0.6102 - accuracy: 0.5743 - val_loss: 0.6243 - val_accuracy: 0.5455
Epoch 225/1000
101/101 [==============================] - 0s 143us/step - loss: 0.6102 - accuracy: 0.5743 - val_loss: 0.6187 - val_accuracy: 0.5455
Epoch 226/1000
101/101 [==============================] - 0s 142us/step - loss: 0.6095 - accuracy: 0.5743 - val_loss: 0.6221 - val_accuracy: 0.5455
Epoch 227/1000
101/101 [==============================] - 0s 141us/step - loss: 0.6096 - accuracy: 0.5743 - val_loss: 0.6199 - val_accuracy: 0.5455
Epoch 228/1000
101/101 [==============================] - 0s 157us/step - loss: 0.6093 - accuracy: 0.5743 - val_loss: 0.6241 - v

...

101/101 [==============================] - 0s 129us/step - loss: 0.4998 - accuracy: 0.7228 - val_loss: 0.5631 - val_accuracy: 0.6591
Epoch 773/1000
101/101 [==============================] - 0s 130us/step - loss: 0.4996 - accuracy: 0.7228 - val_loss: 0.5668 - val_accuracy: 0.6591
Epoch 774/1000
101/101 [==============================] - 0s 128us/step - loss: 0.4998 - accuracy: 0.7228 - val_loss: 0.5566 - val_accuracy: 0.6591
Epoch 775/1000
101/101 [==============================] - 0s 127us/step - loss: 0.5024 - accuracy: 0.7228 - val_loss: 0.5591 - val_accuracy: 0.6591
Epoch 776/1000
101/101 [==============================] - 0s 172us/step - loss: 0.5009 - accuracy: 0.7228 - val_loss: 0.5615 - val_accuracy: 0.6591
Epoch 777/1000
101/101 [==============================] - 0s 151us/step - loss: 0.4999 - accuracy: 0.7228 - val_loss: 0.5647 - val_accuracy: 0.6591
Epoch 778/1000
101/101 [==============================] - 0s 125us/step - loss: 0.4997 - accuracy: 0.7228 - val_loss: 0.5609 - 

101/101 [==============================] - 0s 144us/step - loss: 0.4980 - accuracy: 0.7228 - val_loss: 0.5628 - val_accuracy: 0.6591
Epoch 828/1000
101/101 [==============================] - 0s 159us/step - loss: 0.4972 - accuracy: 0.7228 - val_loss: 0.5679 - val_accuracy: 0.6591
Epoch 829/1000
101/101 [==============================] - 0s 162us/step - loss: 0.4972 - accuracy: 0.7228 - val_loss: 0.5668 - val_accuracy: 0.6591
Epoch 830/1000
101/101 [==============================] - 0s 168us/step - loss: 0.4966 - accuracy: 0.7228 - val_loss: 0.5674 - val_accuracy: 0.6591
Epoch 831/1000
101/101 [==============================] - 0s 117us/step - loss: 0.4967 - accuracy: 0.7228 - val_loss: 0.5740 - val_accuracy: 0.6591
Epoch 832/1000
101/101 [==============================] - 0s 152us/step - loss: 0.4980 - accuracy: 0.7228 - val_loss: 0.5612 - val_accuracy: 0.6591
Epoch 833/1000
101/101 [==============================] - 0s 147us/step - loss: 0.4974 - accuracy: 0.7228 - val_loss: 0.5657 - 

101/101 [==============================] - 0s 126us/step - loss: 0.4958 - accuracy: 0.7228 - val_loss: 0.5668 - val_accuracy: 0.6591
Epoch 883/1000
101/101 [==============================] - 0s 128us/step - loss: 0.4944 - accuracy: 0.7228 - val_loss: 0.5686 - val_accuracy: 0.6591
Epoch 884/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4951 - accuracy: 0.7228 - val_loss: 0.5695 - val_accuracy: 0.6591
Epoch 885/1000
101/101 [==============================] - 0s 145us/step - loss: 0.4961 - accuracy: 0.7228 - val_loss: 0.5693 - val_accuracy: 0.6591
Epoch 886/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4947 - accuracy: 0.7228 - val_loss: 0.5687 - val_accuracy: 0.6591
Epoch 887/1000
101/101 [==============================] - 0s 134us/step - loss: 0.4945 - accuracy: 0.7228 - val_loss: 0.5681 - val_accuracy: 0.6591
Epoch 888/1000
101/101 [==============================] - 0s 140us/step - loss: 0.4944 - accuracy: 0.7228 - val_loss: 0.5679 - 

101/101 [==============================] - 0s 130us/step - loss: 0.4928 - accuracy: 0.7228 - val_loss: 0.5611 - val_accuracy: 0.6591
Epoch 938/1000
101/101 [==============================] - 0s 118us/step - loss: 0.4939 - accuracy: 0.7228 - val_loss: 0.5636 - val_accuracy: 0.6591
Epoch 939/1000
101/101 [==============================] - 0s 160us/step - loss: 0.4934 - accuracy: 0.7228 - val_loss: 0.5672 - val_accuracy: 0.6591
Epoch 940/1000
101/101 [==============================] - 0s 150us/step - loss: 0.4928 - accuracy: 0.7228 - val_loss: 0.5689 - val_accuracy: 0.6591
Epoch 941/1000
101/101 [==============================] - 0s 147us/step - loss: 0.4927 - accuracy: 0.7228 - val_loss: 0.5680 - val_accuracy: 0.6591
Epoch 942/1000
101/101 [==============================] - 0s 120us/step - loss: 0.4930 - accuracy: 0.7228 - val_loss: 0.5672 - val_accuracy: 0.6591
Epoch 943/1000
101/101 [==============================] - 0s 130us/step - loss: 0.4927 - accuracy: 0.7228 - val_loss: 0.5705 - 

101/101 [==============================] - 0s 132us/step - loss: 0.4920 - accuracy: 0.7228 - val_loss: 0.5615 - val_accuracy: 0.6591
Epoch 993/1000
101/101 [==============================] - 0s 93us/step - loss: 0.4914 - accuracy: 0.7228 - val_loss: 0.5651 - val_accuracy: 0.6591
Epoch 994/1000
101/101 [==============================] - 0s 102us/step - loss: 0.4910 - accuracy: 0.7228 - val_loss: 0.5662 - val_accuracy: 0.6591
Epoch 995/1000
101/101 [==============================] - 0s 139us/step - loss: 0.4912 - accuracy: 0.7228 - val_loss: 0.5673 - val_accuracy: 0.6591
Epoch 996/1000
101/101 [==============================] - 0s 138us/step - loss: 0.4914 - accuracy: 0.7228 - val_loss: 0.5632 - val_accuracy: 0.6591
Epoch 997/1000
101/101 [==============================] - 0s 159us/step - loss: 0.4910 - accuracy: 0.7228 - val_loss: 0.5664 - val_accuracy: 0.6591
Epoch 998/1000
101/101 [==============================] - 0s 160us/step - loss: 0.4912 - accuracy: 0.7228 - val_loss: 0.5665 - v

In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

44/44 [==============================] - 0s 161us/step
combination test accuracy: 65.91%


In [24]:
#### improve neural network on combination data
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [25]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 101 samples, validate on 44 samples
Epoch 1/1000
101/101 [==============================] - 0s 1ms/step - loss: 7.4749 - accuracy: 0.4950 - val_loss: 2.2417 - val_accuracy: 0.4545
Epoch 2/1000
101/101 [==============================] - 0s 110us/step - loss: 6.3331 - accuracy: 0.4950 - val_loss: 0.7244 - val_accuracy: 0.4773
Epoch 3/1000
101/101 [==============================] - 0s 218us/step - loss: 0.7866 - accuracy: 0.4257 - val_loss: 0.7090 - val_accuracy: 0.4773
Epoch 4/1000
101/101 [==============================] - 0s 163us/step - loss: 0.6713 - accuracy: 0.4752 - val_loss: 0.8182 - val_accuracy: 0.3864
Epoch 5/1000
101/101 [==============================] - 0s 166us/step - loss: 0.6657 - accuracy: 0.4950 - val_loss: 0.8103 - val_accuracy: 0.3864
Epoch 6/1000
101/101 [==============================] - 0s 178us/step - loss: 0.6613 - accuracy: 0.4950 - val_loss: 0.7978 - val_accuracy: 0.3864
Epoch 7/1000
101/101 [==============================] - 0s 170us/step - loss: 0.6

101/101 [==============================] - 0s 136us/step - loss: 0.5858 - accuracy: 0.6832 - val_loss: 0.7103 - val_accuracy: 0.5455
Epoch 57/1000
101/101 [==============================] - 0s 128us/step - loss: 0.5824 - accuracy: 0.6832 - val_loss: 0.6999 - val_accuracy: 0.5455
Epoch 58/1000
101/101 [==============================] - 0s 111us/step - loss: 0.5825 - accuracy: 0.6832 - val_loss: 0.6614 - val_accuracy: 0.6364
Epoch 59/1000
101/101 [==============================] - 0s 113us/step - loss: 0.5906 - accuracy: 0.7030 - val_loss: 0.6940 - val_accuracy: 0.5455
Epoch 60/1000
101/101 [==============================] - 0s 118us/step - loss: 0.5834 - accuracy: 0.6832 - val_loss: 0.7718 - val_accuracy: 0.5455
Epoch 61/1000
101/101 [==============================] - 0s 114us/step - loss: 0.5845 - accuracy: 0.6832 - val_loss: 0.7282 - val_accuracy: 0.5455
Epoch 62/1000
101/101 [==============================] - 0s 121us/step - loss: 0.5777 - accuracy: 0.6832 - val_loss: 0.7355 - val_ac

Epoch 112/1000
101/101 [==============================] - 0s 249us/step - loss: 0.5427 - accuracy: 0.6832 - val_loss: 0.7607 - val_accuracy: 0.5455
Epoch 113/1000
101/101 [==============================] - 0s 209us/step - loss: 0.5361 - accuracy: 0.6832 - val_loss: 0.8074 - val_accuracy: 0.5455
Epoch 114/1000
101/101 [==============================] - 0s 183us/step - loss: 0.5393 - accuracy: 0.6832 - val_loss: 0.6826 - val_accuracy: 0.5455
Epoch 115/1000
101/101 [==============================] - 0s 224us/step - loss: 0.5332 - accuracy: 0.6832 - val_loss: 0.8031 - val_accuracy: 0.5455
Epoch 116/1000
101/101 [==============================] - 0s 152us/step - loss: 0.5413 - accuracy: 0.6832 - val_loss: 0.6700 - val_accuracy: 0.6136
Epoch 117/1000
101/101 [==============================] - 0s 178us/step - loss: 0.5269 - accuracy: 0.7327 - val_loss: 0.8314 - val_accuracy: 0.5455
Epoch 118/1000
101/101 [==============================] - 0s 187us/step - loss: 0.5437 - accuracy: 0.6832 - val_

101/101 [==============================] - 0s 139us/step - loss: 0.4927 - accuracy: 0.7525 - val_loss: 0.6424 - val_accuracy: 0.6364
Epoch 168/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4835 - accuracy: 0.7723 - val_loss: 0.7270 - val_accuracy: 0.5455
Epoch 169/1000
101/101 [==============================] - 0s 132us/step - loss: 0.4865 - accuracy: 0.7525 - val_loss: 0.5663 - val_accuracy: 0.6591
Epoch 170/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4985 - accuracy: 0.7525 - val_loss: 0.6532 - val_accuracy: 0.6364
Epoch 171/1000
101/101 [==============================] - 0s 125us/step - loss: 0.4744 - accuracy: 0.7822 - val_loss: 0.6876 - val_accuracy: 0.5455
Epoch 172/1000
101/101 [==============================] - 0s 119us/step - loss: 0.4775 - accuracy: 0.7525 - val_loss: 0.6609 - val_accuracy: 0.6364
Epoch 173/1000
101/101 [==============================] - 0s 141us/step - loss: 0.4760 - accuracy: 0.7525 - val_loss: 0.6423 - 

101/101 [==============================] - 0s 231us/step - loss: 0.4375 - accuracy: 0.7723 - val_loss: 0.6543 - val_accuracy: 0.6591
Epoch 223/1000
101/101 [==============================] - ETA: 0s - loss: 0.4331 - accuracy: 0.75 - 0s 213us/step - loss: 0.4331 - accuracy: 0.7822 - val_loss: 0.7507 - val_accuracy: 0.5455
Epoch 224/1000
101/101 [==============================] - 0s 228us/step - loss: 0.5889 - accuracy: 0.6832 - val_loss: 0.9347 - val_accuracy: 0.5909
Epoch 225/1000
101/101 [==============================] - 0s 260us/step - loss: 0.4671 - accuracy: 0.7525 - val_loss: 0.6830 - val_accuracy: 0.6591
Epoch 226/1000
101/101 [==============================] - 0s 229us/step - loss: 0.4355 - accuracy: 0.7624 - val_loss: 0.5366 - val_accuracy: 0.6818
Epoch 227/1000
101/101 [==============================] - 0s 269us/step - loss: 0.4665 - accuracy: 0.7624 - val_loss: 0.6884 - val_accuracy: 0.6364
Epoch 228/1000
101/101 [==============================] - 0s 223us/step - loss: 0.435

...

101/101 [==============================] - 0s 145us/step - loss: 0.2892 - accuracy: 0.8317 - val_loss: 0.5162 - val_accuracy: 0.7045
Epoch 773/1000
101/101 [==============================] - 0s 135us/step - loss: 0.2937 - accuracy: 0.8119 - val_loss: 0.4658 - val_accuracy: 0.7045
Epoch 774/1000
101/101 [==============================] - 0s 154us/step - loss: 0.2948 - accuracy: 0.8119 - val_loss: 0.6246 - val_accuracy: 0.7500
Epoch 775/1000
101/101 [==============================] - 0s 139us/step - loss: 0.2943 - accuracy: 0.8119 - val_loss: 0.6345 - val_accuracy: 0.7045
Epoch 776/1000
101/101 [==============================] - 0s 154us/step - loss: 0.2949 - accuracy: 0.8119 - val_loss: 2.2088 - val_accuracy: 0.6591
Epoch 777/1000
101/101 [==============================] - 0s 159us/step - loss: 0.5525 - accuracy: 0.7822 - val_loss: 0.6422 - val_accuracy: 0.7045
Epoch 778/1000
101/101 [==============================] - 0s 131us/step - loss: 0.2907 - accuracy: 0.8218 - val_loss: 0.5458 - 

101/101 [==============================] - 0s 152us/step - loss: 0.2915 - accuracy: 0.8218 - val_loss: 0.3553 - val_accuracy: 0.7045
Epoch 828/1000
101/101 [==============================] - 0s 136us/step - loss: 0.3008 - accuracy: 0.8119 - val_loss: 0.6119 - val_accuracy: 0.7500
Epoch 829/1000
101/101 [==============================] - 0s 153us/step - loss: 0.2887 - accuracy: 0.8317 - val_loss: 0.4688 - val_accuracy: 0.7500
Epoch 830/1000
101/101 [==============================] - 0s 178us/step - loss: 0.2912 - accuracy: 0.8218 - val_loss: 0.6496 - val_accuracy: 0.7500
Epoch 831/1000
101/101 [==============================] - 0s 137us/step - loss: 0.2874 - accuracy: 0.8317 - val_loss: 0.5176 - val_accuracy: 0.7500
Epoch 832/1000
101/101 [==============================] - 0s 138us/step - loss: 0.2897 - accuracy: 0.8218 - val_loss: 0.5340 - val_accuracy: 0.7500
Epoch 833/1000
101/101 [==============================] - 0s 132us/step - loss: 0.2882 - accuracy: 0.8218 - val_loss: 0.4750 - 

101/101 [==============================] - 0s 135us/step - loss: 0.2856 - accuracy: 0.8317 - val_loss: 0.5440 - val_accuracy: 0.7045
Epoch 883/1000
101/101 [==============================] - 0s 123us/step - loss: 0.2863 - accuracy: 0.8218 - val_loss: 0.4178 - val_accuracy: 0.7045
Epoch 884/1000
101/101 [==============================] - 0s 147us/step - loss: 0.2973 - accuracy: 0.8218 - val_loss: 0.4749 - val_accuracy: 0.7045
Epoch 885/1000
101/101 [==============================] - 0s 119us/step - loss: 0.2879 - accuracy: 0.8119 - val_loss: 0.6748 - val_accuracy: 0.7500
Epoch 886/1000
101/101 [==============================] - 0s 120us/step - loss: 0.2906 - accuracy: 0.8218 - val_loss: 0.6207 - val_accuracy: 0.7500
Epoch 887/1000
101/101 [==============================] - 0s 149us/step - loss: 0.2832 - accuracy: 0.8317 - val_loss: 0.5482 - val_accuracy: 0.7500
Epoch 888/1000
101/101 [==============================] - 0s 156us/step - loss: 0.2865 - accuracy: 0.8119 - val_loss: 0.4804 - 

101/101 [==============================] - 0s 136us/step - loss: 0.2885 - accuracy: 0.8119 - val_loss: 0.4800 - val_accuracy: 0.7045
Epoch 938/1000
101/101 [==============================] - 0s 126us/step - loss: 0.2882 - accuracy: 0.8218 - val_loss: 0.4380 - val_accuracy: 0.7045
Epoch 939/1000
101/101 [==============================] - 0s 155us/step - loss: 0.2939 - accuracy: 0.8218 - val_loss: 0.5001 - val_accuracy: 0.7500
Epoch 940/1000
101/101 [==============================] - 0s 136us/step - loss: 0.2839 - accuracy: 0.8218 - val_loss: 0.6673 - val_accuracy: 0.7500
Epoch 941/1000
101/101 [==============================] - 0s 133us/step - loss: 0.2820 - accuracy: 0.8317 - val_loss: 0.5841 - val_accuracy: 0.7500
Epoch 942/1000
101/101 [==============================] - 0s 146us/step - loss: 0.2821 - accuracy: 0.8218 - val_loss: 2.0026 - val_accuracy: 0.6591
Epoch 943/1000
101/101 [==============================] - 0s 137us/step - loss: 0.5035 - accuracy: 0.8020 - val_loss: 0.6201 - 

101/101 [==============================] - 0s 151us/step - loss: 0.2778 - accuracy: 0.8317 - val_loss: 0.5181 - val_accuracy: 0.7500
Epoch 993/1000
101/101 [==============================] - 0s 145us/step - loss: 0.2832 - accuracy: 0.8218 - val_loss: 0.5975 - val_accuracy: 0.7500
Epoch 994/1000
101/101 [==============================] - 0s 123us/step - loss: 0.2795 - accuracy: 0.8317 - val_loss: 0.6040 - val_accuracy: 0.7500
Epoch 995/1000
101/101 [==============================] - 0s 129us/step - loss: 0.2778 - accuracy: 0.8317 - val_loss: 0.5812 - val_accuracy: 0.7500
Epoch 996/1000
101/101 [==============================] - 0s 131us/step - loss: 0.2785 - accuracy: 0.8317 - val_loss: 0.5739 - val_accuracy: 0.7500
Epoch 997/1000
101/101 [==============================] - 0s 136us/step - loss: 0.2781 - accuracy: 0.8317 - val_loss: 0.7248 - val_accuracy: 0.7500
Epoch 998/1000
101/101 [==============================] - 0s 158us/step - loss: 0.2801 - accuracy: 0.8317 - val_loss: 0.4849 - 

In [27]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

44/44 [==============================] - 0s 196us/step
combination test accuracy: 75.00%


In [28]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [29]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [30]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/1000
242/242 [==============================] - 0s 413us/step - loss: 14.2581 - accuracy: 0.5207 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 2/1000
242/242 [==============================] - 0s 83us/step - loss: 0.6970 - accuracy: 0.5165 - val_loss: 0.6956 - val_accuracy: 0.5000
Epoch 3/1000
242/242 [==============================] - 0s 106us/step - loss: 0.7023 - accuracy: 0.5331 - val_loss: 0.6924 - val_accuracy: 0.5769
Epoch 4/1000
242/242 [==============================] - 0s 108us/step - loss: 0.7285 - accuracy: 0.4959 - val_loss: 0.6929 - val_accuracy: 0.5288
Epoch 5/1000
242/242 [==============================] - 0s 110us/step - loss: 0.6949 - accuracy: 0.4876 - val_loss: 0.6945 - val_accuracy: 0.5096
Epoch 6/1000
242/242 [==============================] - 0s 123us/step - loss: 0.6909 - accuracy: 0.5083 - val_loss: 0.6983 - val_accuracy: 0.5192
Epoch 7/1000
242/242 [==============================] - 0s 113us/step - loss: 

242/242 [==============================] - 0s 127us/step - loss: 0.6657 - accuracy: 0.6281 - val_loss: 0.6877 - val_accuracy: 0.5385
Epoch 57/1000
242/242 [==============================] - 0s 129us/step - loss: 0.6657 - accuracy: 0.6405 - val_loss: 0.6907 - val_accuracy: 0.5385
Epoch 58/1000
242/242 [==============================] - 0s 119us/step - loss: 0.6578 - accuracy: 0.6322 - val_loss: 0.6865 - val_accuracy: 0.5385
Epoch 59/1000
242/242 [==============================] - 0s 115us/step - loss: 0.6622 - accuracy: 0.6281 - val_loss: 0.6831 - val_accuracy: 0.5385
Epoch 60/1000
242/242 [==============================] - 0s 135us/step - loss: 0.6602 - accuracy: 0.6405 - val_loss: 0.6839 - val_accuracy: 0.5385
Epoch 61/1000
242/242 [==============================] - 0s 107us/step - loss: 0.6611 - accuracy: 0.6364 - val_loss: 0.6802 - val_accuracy: 0.5481
Epoch 62/1000
242/242 [==============================] - 0s 131us/step - loss: 0.6581 - accuracy: 0.6322 - val_loss: 0.6779 - val_ac

242/242 [==============================] - 0s 75us/step - loss: 0.6508 - accuracy: 0.6240 - val_loss: 0.6586 - val_accuracy: 0.6154
Epoch 113/1000
242/242 [==============================] - 0s 89us/step - loss: 0.6320 - accuracy: 0.6446 - val_loss: 0.6934 - val_accuracy: 0.5481
Epoch 114/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6411 - accuracy: 0.6281 - val_loss: 0.6596 - val_accuracy: 0.5769
Epoch 115/1000
242/242 [==============================] - 0s 93us/step - loss: 0.6269 - accuracy: 0.6446 - val_loss: 0.6841 - val_accuracy: 0.5481
Epoch 116/1000
242/242 [==============================] - 0s 84us/step - loss: 0.6307 - accuracy: 0.6364 - val_loss: 0.6626 - val_accuracy: 0.5481
Epoch 117/1000
242/242 [==============================] - 0s 84us/step - loss: 0.6322 - accuracy: 0.6240 - val_loss: 0.6671 - val_accuracy: 0.5481
Epoch 118/1000
242/242 [==============================] - 0s 97us/step - loss: 0.6310 - accuracy: 0.6405 - val_loss: 0.6559 - val_acc

Epoch 168/1000
242/242 [==============================] - 0s 79us/step - loss: 0.6227 - accuracy: 0.6488 - val_loss: 0.6816 - val_accuracy: 0.5481
Epoch 169/1000
242/242 [==============================] - 0s 87us/step - loss: 0.6326 - accuracy: 0.6157 - val_loss: 0.6414 - val_accuracy: 0.6154
Epoch 170/1000
242/242 [==============================] - 0s 96us/step - loss: 0.6108 - accuracy: 0.6529 - val_loss: 0.6415 - val_accuracy: 0.6154
Epoch 171/1000
242/242 [==============================] - 0s 78us/step - loss: 0.6041 - accuracy: 0.6570 - val_loss: 0.6552 - val_accuracy: 0.5481
Epoch 172/1000
242/242 [==============================] - 0s 85us/step - loss: 0.6117 - accuracy: 0.6364 - val_loss: 0.6999 - val_accuracy: 0.5769
Epoch 173/1000
242/242 [==============================] - 0s 80us/step - loss: 0.6278 - accuracy: 0.6653 - val_loss: 0.6417 - val_accuracy: 0.5962
Epoch 174/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6333 - accuracy: 0.6405 - val_loss: 0

Epoch 224/1000
242/242 [==============================] - 0s 82us/step - loss: 0.6173 - accuracy: 0.6612 - val_loss: 0.6392 - val_accuracy: 0.5288
Epoch 225/1000
242/242 [==============================] - 0s 92us/step - loss: 0.6144 - accuracy: 0.6612 - val_loss: 0.7901 - val_accuracy: 0.5962
Epoch 226/1000
242/242 [==============================] - 0s 89us/step - loss: 0.6479 - accuracy: 0.6405 - val_loss: 0.6296 - val_accuracy: 0.6058
Epoch 227/1000
242/242 [==============================] - 0s 79us/step - loss: 0.5941 - accuracy: 0.6694 - val_loss: 0.6495 - val_accuracy: 0.5288
Epoch 228/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6237 - accuracy: 0.6322 - val_loss: 0.6314 - val_accuracy: 0.5865
Epoch 229/1000
242/242 [==============================] - 0s 73us/step - loss: 0.5991 - accuracy: 0.6694 - val_loss: 0.6341 - val_accuracy: 0.5577
Epoch 230/1000
242/242 [==============================] - 0s 89us/step - loss: 0.5881 - accuracy: 0.6529 - val_loss: 0

...

Epoch 727/1000
242/242 [==============================] - 0s 77us/step - loss: 0.5501 - accuracy: 0.6818 - val_loss: 0.6169 - val_accuracy: 0.6154
Epoch 728/1000
242/242 [==============================] - 0s 95us/step - loss: 0.5349 - accuracy: 0.7149 - val_loss: 0.6212 - val_accuracy: 0.5385
Epoch 729/1000
242/242 [==============================] - 0s 86us/step - loss: 0.5434 - accuracy: 0.6901 - val_loss: 0.6268 - val_accuracy: 0.6154
Epoch 730/1000
242/242 [==============================] - 0s 87us/step - loss: 0.5386 - accuracy: 0.7149 - val_loss: 0.6097 - val_accuracy: 0.5962
Epoch 731/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5602 - accuracy: 0.6777 - val_loss: 0.6114 - val_accuracy: 0.6058
Epoch 732/1000
242/242 [==============================] - 0s 78us/step - loss: 0.5265 - accuracy: 0.7149 - val_loss: 0.6718 - val_accuracy: 0.5673
Epoch 733/1000
242/242 [==============================] - 0s 81us/step - loss: 0.5680 - accuracy: 0.6818 - val_loss: 0

Epoch 783/1000
242/242 [==============================] - 0s 96us/step - loss: 0.5451 - accuracy: 0.7107 - val_loss: 0.6189 - val_accuracy: 0.6154
Epoch 784/1000
242/242 [==============================] - 0s 84us/step - loss: 0.5272 - accuracy: 0.7149 - val_loss: 0.6036 - val_accuracy: 0.6058
Epoch 785/1000
242/242 [==============================] - 0s 82us/step - loss: 0.5894 - accuracy: 0.6942 - val_loss: 0.6090 - val_accuracy: 0.5962
Epoch 786/1000
242/242 [==============================] - 0s 81us/step - loss: 0.5319 - accuracy: 0.7107 - val_loss: 0.6121 - val_accuracy: 0.6058
Epoch 787/1000
242/242 [==============================] - 0s 82us/step - loss: 0.5317 - accuracy: 0.7025 - val_loss: 0.6128 - val_accuracy: 0.5962
Epoch 788/1000
242/242 [==============================] - 0s 79us/step - loss: 0.5348 - accuracy: 0.7149 - val_loss: 0.6087 - val_accuracy: 0.5962
Epoch 789/1000
242/242 [==============================] - 0s 87us/step - loss: 0.5329 - accuracy: 0.7149 - val_loss: 0

242/242 [==============================] - 0s 82us/step - loss: 0.5182 - accuracy: 0.7025 - val_loss: 0.6266 - val_accuracy: 0.5385
Epoch 839/1000
242/242 [==============================] - 0s 94us/step - loss: 0.5651 - accuracy: 0.6901 - val_loss: 0.6076 - val_accuracy: 0.5865
Epoch 840/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5253 - accuracy: 0.6942 - val_loss: 0.6083 - val_accuracy: 0.5962
Epoch 841/1000
242/242 [==============================] - 0s 90us/step - loss: 0.5343 - accuracy: 0.6818 - val_loss: 0.6190 - val_accuracy: 0.6058
Epoch 842/1000
242/242 [==============================] - 0s 84us/step - loss: 0.5296 - accuracy: 0.6983 - val_loss: 0.6118 - val_accuracy: 0.5962
Epoch 843/1000
242/242 [==============================] - 0s 78us/step - loss: 0.5234 - accuracy: 0.7066 - val_loss: 0.6203 - val_accuracy: 0.6154
Epoch 844/1000
242/242 [==============================] - 0s 83us/step - loss: 0.5248 - accuracy: 0.7107 - val_loss: 0.6087 - val_acc

Epoch 894/1000
242/242 [==============================] - 0s 86us/step - loss: 0.5413 - accuracy: 0.6942 - val_loss: 0.6132 - val_accuracy: 0.5865
Epoch 895/1000
242/242 [==============================] - 0s 93us/step - loss: 0.5221 - accuracy: 0.7025 - val_loss: 0.6161 - val_accuracy: 0.5673
Epoch 896/1000
242/242 [==============================] - 0s 91us/step - loss: 0.5169 - accuracy: 0.7066 - val_loss: 0.6082 - val_accuracy: 0.5962
Epoch 897/1000
242/242 [==============================] - 0s 82us/step - loss: 0.5210 - accuracy: 0.7107 - val_loss: 0.6037 - val_accuracy: 0.5962
Epoch 898/1000
242/242 [==============================] - 0s 81us/step - loss: 0.5190 - accuracy: 0.7066 - val_loss: 0.6118 - val_accuracy: 0.5962
Epoch 899/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5136 - accuracy: 0.7025 - val_loss: 0.6133 - val_accuracy: 0.6154
Epoch 900/1000
242/242 [==============================] - 0s 83us/step - loss: 0.5179 - accuracy: 0.7149 - val_loss: 0

Epoch 950/1000
242/242 [==============================] - 0s 93us/step - loss: 0.5134 - accuracy: 0.6942 - val_loss: 0.6153 - val_accuracy: 0.6154
Epoch 951/1000
242/242 [==============================] - 0s 93us/step - loss: 0.5222 - accuracy: 0.7107 - val_loss: 0.6059 - val_accuracy: 0.6538
Epoch 952/1000
242/242 [==============================] - 0s 89us/step - loss: 0.5151 - accuracy: 0.7273 - val_loss: 0.6054 - val_accuracy: 0.6635
Epoch 953/1000
242/242 [==============================] - 0s 81us/step - loss: 0.5134 - accuracy: 0.7149 - val_loss: 0.6064 - val_accuracy: 0.5865
Epoch 954/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5121 - accuracy: 0.7190 - val_loss: 0.6303 - val_accuracy: 0.5385
Epoch 955/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5168 - accuracy: 0.6983 - val_loss: 0.6105 - val_accuracy: 0.6635
Epoch 956/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5227 - accuracy: 0.6942 - val_loss: 0

In [31]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

104/104 [==============================] - 0s 92us/step
over-sampling test accuracy: 59.62%


In [33]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/1000
242/242 [==============================] - 0s 474us/step - loss: 10.2006 - accuracy: 0.5165 - val_loss: 0.9269 - val_accuracy: 0.5096
Epoch 2/1000
242/242 [==============================] - 0s 83us/step - loss: 0.8166 - accuracy: 0.5207 - val_loss: 0.6973 - val_accuracy: 0.4712
Epoch 3/1000
242/242 [==============================] - 0s 123us/step - loss: 0.6890 - accuracy: 0.5579 - val_loss: 0.6923 - val_accuracy: 0.5288
Epoch 4/1000
242/242 [==============================] - 0s 106us/step - loss: 0.6814 - accuracy: 0.5413 - val_loss: 0.6893 - val_accuracy: 0.4904
Epoch 5/1000
242/242 [==============================] - 0s 129us/step - loss: 0.6799 - accuracy: 0.5331 - val_loss: 0.6899 - val_accuracy: 0.4808
Epoch 6/1000
242/242 [==============================] - 0s 123us/step - loss: 0.6729 - accuracy: 0.5909 - val_loss: 0.6843 - val_accuracy: 0.5385
Epoch 7/1000
242/242 [==============================] - 0s 123us/step - loss: 

242/242 [==============================] - 0s 109us/step - loss: 0.6345 - accuracy: 0.6240 - val_loss: 0.6576 - val_accuracy: 0.5481
Epoch 57/1000
242/242 [==============================] - 0s 113us/step - loss: 0.6334 - accuracy: 0.6198 - val_loss: 0.6612 - val_accuracy: 0.5288
Epoch 58/1000
242/242 [==============================] - 0s 88us/step - loss: 0.6313 - accuracy: 0.6281 - val_loss: 0.6541 - val_accuracy: 0.5865
Epoch 59/1000
242/242 [==============================] - 0s 109us/step - loss: 0.6375 - accuracy: 0.6281 - val_loss: 0.6734 - val_accuracy: 0.5192
Epoch 60/1000
242/242 [==============================] - 0s 90us/step - loss: 0.6374 - accuracy: 0.6198 - val_loss: 0.6536 - val_accuracy: 0.5962
Epoch 61/1000
242/242 [==============================] - 0s 95us/step - loss: 0.6366 - accuracy: 0.6240 - val_loss: 0.6589 - val_accuracy: 0.5288
Epoch 62/1000
242/242 [==============================] - 0s 109us/step - loss: 0.6356 - accuracy: 0.6446 - val_loss: 0.6655 - val_accur

Epoch 112/1000
242/242 [==============================] - 0s 80us/step - loss: 0.6117 - accuracy: 0.6529 - val_loss: 0.6721 - val_accuracy: 0.5288
Epoch 113/1000
242/242 [==============================] - 0s 90us/step - loss: 0.6070 - accuracy: 0.6405 - val_loss: 0.6541 - val_accuracy: 0.5288
Epoch 114/1000
242/242 [==============================] - 0s 99us/step - loss: 0.5981 - accuracy: 0.6529 - val_loss: 0.6329 - val_accuracy: 0.5673
Epoch 115/1000
242/242 [==============================] - 0s 98us/step - loss: 0.6165 - accuracy: 0.6570 - val_loss: 0.6511 - val_accuracy: 0.5481
Epoch 116/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6190 - accuracy: 0.6281 - val_loss: 0.6447 - val_accuracy: 0.5096
Epoch 117/1000
242/242 [==============================] - 0s 83us/step - loss: 0.6019 - accuracy: 0.6529 - val_loss: 0.6568 - val_accuracy: 0.5096
Epoch 118/1000
242/242 [==============================] - 0s 85us/step - loss: 0.6169 - accuracy: 0.6653 - val_loss: 0

242/242 [==============================] - 0s 92us/step - loss: 0.6121 - accuracy: 0.6446 - val_loss: 0.6452 - val_accuracy: 0.5385
Epoch 168/1000
242/242 [==============================] - 0s 100us/step - loss: 0.6102 - accuracy: 0.6240 - val_loss: 0.6237 - val_accuracy: 0.5673
Epoch 169/1000
242/242 [==============================] - 0s 113us/step - loss: 0.5903 - accuracy: 0.6860 - val_loss: 0.6402 - val_accuracy: 0.5385
Epoch 170/1000
242/242 [==============================] - 0s 101us/step - loss: 0.5904 - accuracy: 0.6694 - val_loss: 0.6346 - val_accuracy: 0.5673
Epoch 171/1000
242/242 [==============================] - 0s 88us/step - loss: 0.5999 - accuracy: 0.6570 - val_loss: 0.6660 - val_accuracy: 0.5385
Epoch 172/1000
242/242 [==============================] - 0s 113us/step - loss: 0.6144 - accuracy: 0.6736 - val_loss: 0.6237 - val_accuracy: 0.5769
Epoch 173/1000
242/242 [==============================] - 0s 145us/step - loss: 0.6272 - accuracy: 0.6240 - val_loss: 0.6452 - va

242/242 [==============================] - 0s 88us/step - loss: 0.6967 - accuracy: 0.6322 - val_loss: 0.6699 - val_accuracy: 0.5481
Epoch 223/1000
242/242 [==============================] - 0s 74us/step - loss: 0.6228 - accuracy: 0.6488 - val_loss: 0.6538 - val_accuracy: 0.5385
Epoch 224/1000
242/242 [==============================] - 0s 89us/step - loss: 0.6076 - accuracy: 0.6570 - val_loss: 0.6381 - val_accuracy: 0.6154
Epoch 225/1000
242/242 [==============================] - 0s 95us/step - loss: 0.6061 - accuracy: 0.6612 - val_loss: 0.6369 - val_accuracy: 0.6058
Epoch 226/1000
242/242 [==============================] - 0s 106us/step - loss: 0.5958 - accuracy: 0.6694 - val_loss: 0.6280 - val_accuracy: 0.5962
Epoch 227/1000
242/242 [==============================] - 0s 100us/step - loss: 0.6023 - accuracy: 0.6818 - val_loss: 0.6439 - val_accuracy: 0.5577
Epoch 228/1000
242/242 [==============================] - 0s 87us/step - loss: 0.5922 - accuracy: 0.6818 - val_loss: 0.6218 - val_a

...

Epoch 722/1000
242/242 [==============================] - 0s 93us/step - loss: 0.5217 - accuracy: 0.6860 - val_loss: 0.5971 - val_accuracy: 0.5865
Epoch 723/1000
242/242 [==============================] - 0s 97us/step - loss: 0.5133 - accuracy: 0.7149 - val_loss: 0.5854 - val_accuracy: 0.6346
Epoch 724/1000
242/242 [==============================] - 0s 92us/step - loss: 0.5308 - accuracy: 0.7107 - val_loss: 0.5901 - val_accuracy: 0.6635
Epoch 725/1000
242/242 [==============================] - 0s 94us/step - loss: 0.5139 - accuracy: 0.7025 - val_loss: 0.5738 - val_accuracy: 0.6442
Epoch 726/1000
242/242 [==============================] - 0s 92us/step - loss: 0.5064 - accuracy: 0.7107 - val_loss: 0.5838 - val_accuracy: 0.6731
Epoch 727/1000
242/242 [==============================] - 0s 91us/step - loss: 0.5183 - accuracy: 0.6942 - val_loss: 0.6137 - val_accuracy: 0.6346
Epoch 728/1000
242/242 [==============================] - 0s 79us/step - loss: 0.5261 - accuracy: 0.6983 - val_loss: 0

Epoch 778/1000
242/242 [==============================] - 0s 104us/step - loss: 0.5046 - accuracy: 0.7149 - val_loss: 0.5807 - val_accuracy: 0.6442
Epoch 779/1000
242/242 [==============================] - 0s 98us/step - loss: 0.5015 - accuracy: 0.7066 - val_loss: 0.5902 - val_accuracy: 0.5865
Epoch 780/1000
242/242 [==============================] - 0s 87us/step - loss: 0.5057 - accuracy: 0.7355 - val_loss: 0.6768 - val_accuracy: 0.5577
Epoch 781/1000
242/242 [==============================] - 0s 84us/step - loss: 0.5015 - accuracy: 0.7438 - val_loss: 0.5719 - val_accuracy: 0.6538
Epoch 782/1000
242/242 [==============================] - 0s 95us/step - loss: 0.4963 - accuracy: 0.7231 - val_loss: 0.5915 - val_accuracy: 0.6346
Epoch 783/1000
242/242 [==============================] - 0s 84us/step - loss: 0.5329 - accuracy: 0.7066 - val_loss: 0.6944 - val_accuracy: 0.5673
Epoch 784/1000
242/242 [==============================] - 0s 88us/step - loss: 0.5246 - accuracy: 0.7190 - val_loss: 

Epoch 834/1000
242/242 [==============================] - 0s 96us/step - loss: 0.4963 - accuracy: 0.7107 - val_loss: 0.5722 - val_accuracy: 0.6635
Epoch 835/1000
242/242 [==============================] - 0s 90us/step - loss: 0.4963 - accuracy: 0.7149 - val_loss: 0.6520 - val_accuracy: 0.6250
Epoch 836/1000
242/242 [==============================] - 0s 96us/step - loss: 0.5000 - accuracy: 0.7149 - val_loss: 0.5680 - val_accuracy: 0.6538
Epoch 837/1000
242/242 [==============================] - 0s 88us/step - loss: 0.4976 - accuracy: 0.7190 - val_loss: 0.6120 - val_accuracy: 0.6250
Epoch 838/1000
242/242 [==============================] - 0s 88us/step - loss: 0.4899 - accuracy: 0.7190 - val_loss: 0.5839 - val_accuracy: 0.6442
Epoch 839/1000
242/242 [==============================] - 0s 77us/step - loss: 0.5014 - accuracy: 0.7149 - val_loss: 0.5683 - val_accuracy: 0.6442
Epoch 840/1000
242/242 [==============================] - 0s 84us/step - loss: 0.5103 - accuracy: 0.7025 - val_loss: 0

Epoch 890/1000
242/242 [==============================] - 0s 83us/step - loss: 0.5029 - accuracy: 0.6983 - val_loss: 0.5744 - val_accuracy: 0.6442
Epoch 891/1000
242/242 [==============================] - 0s 97us/step - loss: 0.4953 - accuracy: 0.7149 - val_loss: 0.5871 - val_accuracy: 0.5962
Epoch 892/1000
242/242 [==============================] - 0s 92us/step - loss: 0.5012 - accuracy: 0.7190 - val_loss: 0.5711 - val_accuracy: 0.6538
Epoch 893/1000
242/242 [==============================] - 0s 91us/step - loss: 0.5003 - accuracy: 0.7025 - val_loss: 0.5658 - val_accuracy: 0.6538
Epoch 894/1000
242/242 [==============================] - 0s 86us/step - loss: 0.4912 - accuracy: 0.7231 - val_loss: 0.5710 - val_accuracy: 0.6538
Epoch 895/1000
242/242 [==============================] - 0s 86us/step - loss: 0.5041 - accuracy: 0.7273 - val_loss: 0.6536 - val_accuracy: 0.5769
Epoch 896/1000
242/242 [==============================] - 0s 91us/step - loss: 0.5533 - accuracy: 0.6901 - val_loss: 0

242/242 [==============================] - 0s 92us/step - loss: 0.4856 - accuracy: 0.7273 - val_loss: 0.5705 - val_accuracy: 0.6538
Epoch 946/1000
242/242 [==============================] - 0s 115us/step - loss: 0.4906 - accuracy: 0.7231 - val_loss: 0.5595 - val_accuracy: 0.7212
Epoch 947/1000
242/242 [==============================] - 0s 95us/step - loss: 0.4956 - accuracy: 0.7107 - val_loss: 0.5733 - val_accuracy: 0.6346
Epoch 948/1000
242/242 [==============================] - 0s 94us/step - loss: 0.4866 - accuracy: 0.7231 - val_loss: 0.5752 - val_accuracy: 0.6346
Epoch 949/1000
242/242 [==============================] - 0s 99us/step - loss: 0.4956 - accuracy: 0.7355 - val_loss: 0.5601 - val_accuracy: 0.6442
Epoch 950/1000
242/242 [==============================] - 0s 83us/step - loss: 0.4801 - accuracy: 0.7190 - val_loss: 0.5588 - val_accuracy: 0.6538
Epoch 951/1000
242/242 [==============================] - 0s 100us/step - loss: 0.4947 - accuracy: 0.7231 - val_loss: 0.5568 - val_a

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

104/104 [==============================] - 0s 94us/step
over-sampling test accuracy: 65.38%


In [41]:
############ Feature selection using lasso ##########

In [37]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [38]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [39]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [40]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [41]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 86
selected features: 19


In [42]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 0,  1,  2,  3,  5,  6,  7,  9, 10, 12, 14, 25, 44, 45, 48, 49,
        50, 84, 85]])

In [43]:
names_arr = np.array(names)
names_arr[cols]

array(['TTGTTATAGTC', 'TTAATTTAATAGA', 'TTAACATAATAAT', 'TGCAATCTCTTTAT',
       'TACATACCGAT', 'GTGTATCATAAT', 'GCTGTTGAAATGGC', 'GAGTCCTGTT',
       'GAGTCCTGTTT', 'CTTTTTCACCTGT', 'CGCCATTATGTT', 'X1_445856_T_C',
       'group_6031', 'group_3025', 'group_2869', 'group_1792', 'clpA',
       'ST', 'CC'], dtype='<U99')

In [44]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([ 4,  8, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28,
            29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 46, 47,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83],
           dtype='int64')

In [45]:
len(removed_features)

67

In [46]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 19) (255,) (255, 20)


In [47]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 76), (1, 69)]


In [48]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 173), (1, 173)]


In [49]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [50]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [51]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [52]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 101 samples, validate on 44 samples
Epoch 1/1000
101/101 [==============================] - 0s 963us/step - loss: 108.9692 - accuracy: 0.5050 - val_loss: 0.7058 - val_accuracy: 0.5000
Epoch 2/1000
101/101 [==============================] - 0s 109us/step - loss: 0.7494 - accuracy: 0.4752 - val_loss: 0.6882 - val_accuracy: 0.5000
Epoch 3/1000
101/101 [==============================] - 0s 157us/step - loss: 0.7162 - accuracy: 0.4752 - val_loss: 0.6844 - val_accuracy: 0.4773
Epoch 4/1000
101/101 [==============================] - 0s 177us/step - loss: 0.6864 - accuracy: 0.4950 - val_loss: 0.6792 - val_accuracy: 0.4773
Epoch 5/1000
101/101 [==============================] - 0s 182us/step - loss: 0.6843 - accuracy: 0.4950 - val_loss: 0.6749 - val_accuracy: 0.4773
Epoch 6/1000
101/101 [==============================] - 0s 193us/step - loss: 0.6826 - accuracy: 0.5050 - val_loss: 0.6783 - val_accuracy: 0.5455
Epoch 7/1000
101/101 [==============================] - 0s 217us/step - loss:

101/101 [==============================] - 0s 186us/step - loss: 0.6395 - accuracy: 0.6436 - val_loss: 0.6558 - val_accuracy: 0.6136
Epoch 57/1000
101/101 [==============================] - 0s 236us/step - loss: 0.6394 - accuracy: 0.6436 - val_loss: 0.6557 - val_accuracy: 0.6136
Epoch 58/1000
101/101 [==============================] - 0s 230us/step - loss: 0.6385 - accuracy: 0.6634 - val_loss: 0.6550 - val_accuracy: 0.6136
Epoch 59/1000
101/101 [==============================] - 0s 168us/step - loss: 0.6382 - accuracy: 0.6634 - val_loss: 0.6546 - val_accuracy: 0.6136
Epoch 60/1000
101/101 [==============================] - 0s 130us/step - loss: 0.6370 - accuracy: 0.6733 - val_loss: 0.6572 - val_accuracy: 0.5455
Epoch 61/1000
101/101 [==============================] - 0s 142us/step - loss: 0.6386 - accuracy: 0.6535 - val_loss: 0.6539 - val_accuracy: 0.6136
Epoch 62/1000
101/101 [==============================] - 0s 174us/step - loss: 0.6361 - accuracy: 0.6733 - val_loss: 0.6551 - val_ac

Epoch 112/1000
101/101 [==============================] - 0s 146us/step - loss: 0.6033 - accuracy: 0.6832 - val_loss: 0.6299 - val_accuracy: 0.6364
Epoch 113/1000
101/101 [==============================] - 0s 159us/step - loss: 0.6010 - accuracy: 0.6832 - val_loss: 0.8634 - val_accuracy: 0.6136
Epoch 114/1000
101/101 [==============================] - 0s 126us/step - loss: 0.6611 - accuracy: 0.6733 - val_loss: 0.6286 - val_accuracy: 0.6364
Epoch 115/1000
101/101 [==============================] - 0s 139us/step - loss: 0.6086 - accuracy: 0.6832 - val_loss: 0.6319 - val_accuracy: 0.6136
Epoch 116/1000
101/101 [==============================] - 0s 133us/step - loss: 0.6072 - accuracy: 0.6733 - val_loss: 0.6303 - val_accuracy: 0.6136
Epoch 117/1000
101/101 [==============================] - 0s 139us/step - loss: 0.6080 - accuracy: 0.6733 - val_loss: 0.6285 - val_accuracy: 0.6136
Epoch 118/1000
101/101 [==============================] - 0s 128us/step - loss: 0.6065 - accuracy: 0.6832 - val_

101/101 [==============================] - 0s 142us/step - loss: 0.5849 - accuracy: 0.6832 - val_loss: 0.6115 - val_accuracy: 0.6364
Epoch 168/1000
101/101 [==============================] - 0s 127us/step - loss: 0.5857 - accuracy: 0.6733 - val_loss: 0.6100 - val_accuracy: 0.6364
Epoch 169/1000
101/101 [==============================] - 0s 150us/step - loss: 0.5839 - accuracy: 0.6832 - val_loss: 0.6154 - val_accuracy: 0.6364
Epoch 170/1000
101/101 [==============================] - 0s 150us/step - loss: 0.5829 - accuracy: 0.6832 - val_loss: 0.6133 - val_accuracy: 0.6364
Epoch 171/1000
101/101 [==============================] - 0s 141us/step - loss: 0.5842 - accuracy: 0.6832 - val_loss: 0.6108 - val_accuracy: 0.6136
Epoch 172/1000
101/101 [==============================] - 0s 140us/step - loss: 0.5809 - accuracy: 0.6733 - val_loss: 0.6073 - val_accuracy: 0.6364
Epoch 173/1000
101/101 [==============================] - 0s 138us/step - loss: 0.5816 - accuracy: 0.6832 - val_loss: 0.6095 - 

101/101 [==============================] - 0s 128us/step - loss: 0.5711 - accuracy: 0.6832 - val_loss: 0.5885 - val_accuracy: 0.6364
Epoch 223/1000
101/101 [==============================] - 0s 154us/step - loss: 0.5631 - accuracy: 0.6832 - val_loss: 0.6001 - val_accuracy: 0.6364
Epoch 224/1000
101/101 [==============================] - 0s 164us/step - loss: 0.5566 - accuracy: 0.6832 - val_loss: 0.5887 - val_accuracy: 0.6364
Epoch 225/1000
101/101 [==============================] - 0s 162us/step - loss: 0.5616 - accuracy: 0.6832 - val_loss: 0.5896 - val_accuracy: 0.6364
Epoch 226/1000
101/101 [==============================] - 0s 170us/step - loss: 0.5593 - accuracy: 0.6832 - val_loss: 0.5967 - val_accuracy: 0.6364
Epoch 227/1000
101/101 [==============================] - 0s 165us/step - loss: 0.5590 - accuracy: 0.6832 - val_loss: 0.5913 - val_accuracy: 0.6136
Epoch 228/1000
101/101 [==============================] - 0s 158us/step - loss: 0.5621 - accuracy: 0.6733 - val_loss: 0.5906 - 

...

101/101 [==============================] - 0s 150us/step - loss: 0.5038 - accuracy: 0.7327 - val_loss: 0.5554 - val_accuracy: 0.7045
Epoch 773/1000
101/101 [==============================] - 0s 140us/step - loss: 0.5038 - accuracy: 0.7327 - val_loss: 0.5549 - val_accuracy: 0.7045
Epoch 774/1000
101/101 [==============================] - 0s 168us/step - loss: 0.5041 - accuracy: 0.7327 - val_loss: 0.5558 - val_accuracy: 0.7045
Epoch 775/1000
101/101 [==============================] - 0s 165us/step - loss: 0.5034 - accuracy: 0.7327 - val_loss: 0.5557 - val_accuracy: 0.7045
Epoch 776/1000
101/101 [==============================] - 0s 135us/step - loss: 0.5032 - accuracy: 0.7327 - val_loss: 0.5551 - val_accuracy: 0.7045
Epoch 777/1000
101/101 [==============================] - 0s 125us/step - loss: 0.5033 - accuracy: 0.7327 - val_loss: 0.5554 - val_accuracy: 0.7045
Epoch 778/1000
101/101 [==============================] - 0s 137us/step - loss: 0.5029 - accuracy: 0.7327 - val_loss: 0.5557 - 

101/101 [==============================] - 0s 154us/step - loss: 0.4984 - accuracy: 0.7327 - val_loss: 0.5678 - val_accuracy: 0.6364
Epoch 828/1000
101/101 [==============================] - 0s 119us/step - loss: 0.4993 - accuracy: 0.7327 - val_loss: 0.5647 - val_accuracy: 0.6364
Epoch 829/1000
101/101 [==============================] - 0s 148us/step - loss: 0.4984 - accuracy: 0.7327 - val_loss: 0.5592 - val_accuracy: 0.6364
Epoch 830/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4981 - accuracy: 0.7327 - val_loss: 0.5571 - val_accuracy: 0.7045
Epoch 831/1000
101/101 [==============================] - 0s 120us/step - loss: 0.4976 - accuracy: 0.7327 - val_loss: 0.5551 - val_accuracy: 0.7045
Epoch 832/1000
101/101 [==============================] - 0s 135us/step - loss: 0.4981 - accuracy: 0.7327 - val_loss: 0.5567 - val_accuracy: 0.7045
Epoch 833/1000
101/101 [==============================] - 0s 135us/step - loss: 0.4992 - accuracy: 0.7525 - val_loss: 0.5551 - 

101/101 [==============================] - 0s 132us/step - loss: 0.4931 - accuracy: 0.7327 - val_loss: 0.5546 - val_accuracy: 0.6364
Epoch 883/1000
101/101 [==============================] - 0s 141us/step - loss: 0.4931 - accuracy: 0.7327 - val_loss: 0.5575 - val_accuracy: 0.6364
Epoch 884/1000
101/101 [==============================] - 0s 165us/step - loss: 0.4928 - accuracy: 0.7525 - val_loss: 0.5556 - val_accuracy: 0.6364
Epoch 885/1000
101/101 [==============================] - 0s 154us/step - loss: 0.4924 - accuracy: 0.7327 - val_loss: 0.5558 - val_accuracy: 0.6364
Epoch 886/1000
101/101 [==============================] - 0s 151us/step - loss: 0.4922 - accuracy: 0.7327 - val_loss: 0.5547 - val_accuracy: 0.6364
Epoch 887/1000
101/101 [==============================] - 0s 132us/step - loss: 0.4930 - accuracy: 0.7327 - val_loss: 0.5552 - val_accuracy: 0.6364
Epoch 888/1000
101/101 [==============================] - 0s 153us/step - loss: 0.4930 - accuracy: 0.7327 - val_loss: 0.5529 - 

101/101 [==============================] - 0s 112us/step - loss: 0.4883 - accuracy: 0.7327 - val_loss: 0.5707 - val_accuracy: 0.6591
Epoch 938/1000
101/101 [==============================] - 0s 117us/step - loss: 0.4909 - accuracy: 0.7525 - val_loss: 0.5714 - val_accuracy: 0.6591
Epoch 939/1000
101/101 [==============================] - 0s 94us/step - loss: 0.4893 - accuracy: 0.7525 - val_loss: 0.5663 - val_accuracy: 0.6591
Epoch 940/1000
101/101 [==============================] - 0s 124us/step - loss: 0.4881 - accuracy: 0.7525 - val_loss: 0.5626 - val_accuracy: 0.6591
Epoch 941/1000
101/101 [==============================] - 0s 91us/step - loss: 0.4879 - accuracy: 0.7525 - val_loss: 0.5624 - val_accuracy: 0.6591
Epoch 942/1000
101/101 [==============================] - 0s 99us/step - loss: 0.4878 - accuracy: 0.7624 - val_loss: 0.5598 - val_accuracy: 0.6591
Epoch 943/1000
101/101 [==============================] - 0s 95us/step - loss: 0.4873 - accuracy: 0.7525 - val_loss: 0.5527 - val_

101/101 [==============================] - 0s 169us/step - loss: 0.4819 - accuracy: 0.7327 - val_loss: 0.5458 - val_accuracy: 0.7045
Epoch 993/1000
101/101 [==============================] - 0s 149us/step - loss: 0.4811 - accuracy: 0.7426 - val_loss: 0.5525 - val_accuracy: 0.6364
Epoch 994/1000
101/101 [==============================] - 0s 148us/step - loss: 0.4825 - accuracy: 0.7327 - val_loss: 0.5505 - val_accuracy: 0.6364
Epoch 995/1000
101/101 [==============================] - 0s 162us/step - loss: 0.4809 - accuracy: 0.7426 - val_loss: 0.5520 - val_accuracy: 0.6364
Epoch 996/1000
101/101 [==============================] - 0s 169us/step - loss: 0.4823 - accuracy: 0.7327 - val_loss: 0.5554 - val_accuracy: 0.6364
Epoch 997/1000
101/101 [==============================] - 0s 179us/step - loss: 0.4819 - accuracy: 0.7525 - val_loss: 0.5568 - val_accuracy: 0.6364
Epoch 998/1000
101/101 [==============================] - 0s 444us/step - loss: 0.4870 - accuracy: 0.7525 - val_loss: 0.5456 - 

In [54]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


44/44 [==============================] - 0s 190us/step
combination test accuracy: 63.64%


In [55]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [56]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 101 samples, validate on 44 samples
Epoch 1/1000
101/101 [==============================] - 0s 1ms/step - loss: 70.3378 - accuracy: 0.5743 - val_loss: 3.0288 - val_accuracy: 0.5455
Epoch 2/1000
101/101 [==============================] - 0s 105us/step - loss: 1.3956 - accuracy: 0.5545 - val_loss: 1.0818 - val_accuracy: 0.5227
Epoch 3/1000
101/101 [==============================] - 0s 152us/step - loss: 1.0665 - accuracy: 0.6436 - val_loss: 1.8073 - val_accuracy: 0.5227
Epoch 4/1000
101/101 [==============================] - 0s 175us/step - loss: 1.2461 - accuracy: 0.5842 - val_loss: 0.8126 - val_accuracy: 0.5227
Epoch 5/1000
101/101 [==============================] - 0s 174us/step - loss: 1.1002 - accuracy: 0.5446 - val_loss: 0.7098 - val_accuracy: 0.5682
Epoch 6/1000
101/101 [==============================] - 0s 173us/step - loss: 0.6818 - accuracy: 0.6535 - val_loss: 0.6671 - val_accuracy: 0.6818
Epoch 7/1000
101/101 [==============================] - 0s 185us/step - loss: 0.

101/101 [==============================] - 0s 130us/step - loss: 0.5984 - accuracy: 0.7228 - val_loss: 0.6294 - val_accuracy: 0.7273
Epoch 57/1000
101/101 [==============================] - 0s 124us/step - loss: 0.6103 - accuracy: 0.7426 - val_loss: 0.6048 - val_accuracy: 0.6364
Epoch 58/1000
101/101 [==============================] - 0s 113us/step - loss: 0.5943 - accuracy: 0.6832 - val_loss: 0.6801 - val_accuracy: 0.5455
Epoch 59/1000
101/101 [==============================] - 0s 89us/step - loss: 0.6267 - accuracy: 0.6832 - val_loss: 0.6071 - val_accuracy: 0.6364
Epoch 60/1000
101/101 [==============================] - 0s 113us/step - loss: 0.5932 - accuracy: 0.7030 - val_loss: 0.6055 - val_accuracy: 0.6364
Epoch 61/1000
101/101 [==============================] - 0s 97us/step - loss: 0.6004 - accuracy: 0.6931 - val_loss: 0.6026 - val_accuracy: 0.6364
Epoch 62/1000
101/101 [==============================] - 0s 105us/step - loss: 0.5979 - accuracy: 0.6832 - val_loss: 0.7486 - val_accu

101/101 [==============================] - 0s 194us/step - loss: 0.5510 - accuracy: 0.7525 - val_loss: 0.6463 - val_accuracy: 0.6364
Epoch 112/1000
101/101 [==============================] - 0s 127us/step - loss: 0.5990 - accuracy: 0.7129 - val_loss: 0.5766 - val_accuracy: 0.7273
Epoch 113/1000
101/101 [==============================] - 0s 139us/step - loss: 0.5577 - accuracy: 0.7228 - val_loss: 0.5745 - val_accuracy: 0.7273
Epoch 114/1000
101/101 [==============================] - 0s 131us/step - loss: 0.5487 - accuracy: 0.7327 - val_loss: 0.5868 - val_accuracy: 0.7273
Epoch 115/1000
101/101 [==============================] - 0s 118us/step - loss: 0.5544 - accuracy: 0.7228 - val_loss: 0.5831 - val_accuracy: 0.7273
Epoch 116/1000
101/101 [==============================] - 0s 124us/step - loss: 0.5437 - accuracy: 0.7525 - val_loss: 0.5701 - val_accuracy: 0.7273
Epoch 117/1000
101/101 [==============================] - 0s 137us/step - loss: 0.5564 - accuracy: 0.7228 - val_loss: 0.5660 - 

101/101 [==============================] - 0s 146us/step - loss: 0.5514 - accuracy: 0.6931 - val_loss: 0.5504 - val_accuracy: 0.7273
Epoch 167/1000
101/101 [==============================] - 0s 154us/step - loss: 0.5130 - accuracy: 0.7723 - val_loss: 0.5428 - val_accuracy: 0.7273
Epoch 168/1000
101/101 [==============================] - 0s 153us/step - loss: 0.5121 - accuracy: 0.7723 - val_loss: 0.7831 - val_accuracy: 0.5455
Epoch 169/1000
101/101 [==============================] - 0s 163us/step - loss: 0.6563 - accuracy: 0.6139 - val_loss: 0.5884 - val_accuracy: 0.6364
Epoch 170/1000
101/101 [==============================] - 0s 160us/step - loss: 0.5546 - accuracy: 0.6931 - val_loss: 0.6824 - val_accuracy: 0.5682
Epoch 171/1000
101/101 [==============================] - 0s 156us/step - loss: 0.5626 - accuracy: 0.7030 - val_loss: 0.6055 - val_accuracy: 0.6364
Epoch 172/1000
101/101 [==============================] - 0s 123us/step - loss: 0.5386 - accuracy: 0.7327 - val_loss: 0.5472 - 

101/101 [==============================] - 0s 156us/step - loss: 0.5928 - accuracy: 0.6832 - val_loss: 0.5723 - val_accuracy: 0.6364
Epoch 222/1000
101/101 [==============================] - 0s 168us/step - loss: 0.5501 - accuracy: 0.6832 - val_loss: 0.5826 - val_accuracy: 0.7273
Epoch 223/1000
101/101 [==============================] - 0s 162us/step - loss: 0.5455 - accuracy: 0.7327 - val_loss: 0.5910 - val_accuracy: 0.7273
Epoch 224/1000
101/101 [==============================] - 0s 194us/step - loss: 0.5474 - accuracy: 0.7426 - val_loss: 0.5703 - val_accuracy: 0.6364
Epoch 225/1000
101/101 [==============================] - 0s 161us/step - loss: 0.5527 - accuracy: 0.7129 - val_loss: 0.5905 - val_accuracy: 0.7273
Epoch 226/1000
101/101 [==============================] - 0s 166us/step - loss: 0.5528 - accuracy: 0.7426 - val_loss: 0.5696 - val_accuracy: 0.6364
Epoch 227/1000
101/101 [==============================] - 0s 193us/step - loss: 0.5409 - accuracy: 0.6832 - val_loss: 0.6007 - 

...

101/101 [==============================] - 0s 134us/step - loss: 0.4816 - accuracy: 0.7327 - val_loss: 0.5594 - val_accuracy: 0.6364
Epoch 772/1000
101/101 [==============================] - 0s 134us/step - loss: 0.4814 - accuracy: 0.7327 - val_loss: 0.5432 - val_accuracy: 0.6364
Epoch 773/1000
101/101 [==============================] - 0s 147us/step - loss: 0.4811 - accuracy: 0.7327 - val_loss: 0.5523 - val_accuracy: 0.6364
Epoch 774/1000
101/101 [==============================] - 0s 164us/step - loss: 0.4813 - accuracy: 0.7327 - val_loss: 0.5440 - val_accuracy: 0.6364
Epoch 775/1000
101/101 [==============================] - 0s 160us/step - loss: 0.4817 - accuracy: 0.7327 - val_loss: 0.5399 - val_accuracy: 0.7045
Epoch 776/1000
101/101 [==============================] - 0s 149us/step - loss: 0.4812 - accuracy: 0.7327 - val_loss: 0.5429 - val_accuracy: 0.6364
Epoch 777/1000
101/101 [==============================] - 0s 139us/step - loss: 0.4808 - accuracy: 0.7327 - val_loss: 0.5410 - 

101/101 [==============================] - 0s 177us/step - loss: 0.4774 - accuracy: 0.7525 - val_loss: 0.5384 - val_accuracy: 0.6591
Epoch 827/1000
101/101 [==============================] - 0s 146us/step - loss: 0.4771 - accuracy: 0.7426 - val_loss: 0.5355 - val_accuracy: 0.7045
Epoch 828/1000
101/101 [==============================] - 0s 142us/step - loss: 0.4771 - accuracy: 0.7327 - val_loss: 0.5390 - val_accuracy: 0.6364
Epoch 829/1000
101/101 [==============================] - 0s 158us/step - loss: 0.4769 - accuracy: 0.7327 - val_loss: 0.5458 - val_accuracy: 0.6591
Epoch 830/1000
101/101 [==============================] - 0s 151us/step - loss: 0.4773 - accuracy: 0.7525 - val_loss: 0.5444 - val_accuracy: 0.6591
Epoch 831/1000
101/101 [==============================] - 0s 134us/step - loss: 0.4772 - accuracy: 0.7525 - val_loss: 0.5434 - val_accuracy: 0.6591
Epoch 832/1000
101/101 [==============================] - 0s 136us/step - loss: 0.4769 - accuracy: 0.7525 - val_loss: 0.5435 - 

101/101 [==============================] - 0s 141us/step - loss: 0.4743 - accuracy: 0.7327 - val_loss: 0.5421 - val_accuracy: 0.6364
Epoch 882/1000
101/101 [==============================] - 0s 127us/step - loss: 0.4734 - accuracy: 0.7327 - val_loss: 0.5366 - val_accuracy: 0.6364
Epoch 883/1000
101/101 [==============================] - 0s 157us/step - loss: 0.4737 - accuracy: 0.7327 - val_loss: 0.5398 - val_accuracy: 0.6591
Epoch 884/1000
101/101 [==============================] - 0s 171us/step - loss: 0.4734 - accuracy: 0.7525 - val_loss: 0.5479 - val_accuracy: 0.6591
Epoch 885/1000
101/101 [==============================] - 0s 128us/step - loss: 0.4732 - accuracy: 0.7525 - val_loss: 0.5478 - val_accuracy: 0.6364
Epoch 886/1000
101/101 [==============================] - 0s 137us/step - loss: 0.4728 - accuracy: 0.7327 - val_loss: 0.5520 - val_accuracy: 0.6364
Epoch 887/1000
101/101 [==============================] - 0s 145us/step - loss: 0.4730 - accuracy: 0.7327 - val_loss: 0.5495 - 

101/101 [==============================] - 0s 146us/step - loss: 0.4730 - accuracy: 0.7525 - val_loss: 0.5531 - val_accuracy: 0.6591
Epoch 937/1000
101/101 [==============================] - 0s 132us/step - loss: 0.4738 - accuracy: 0.7525 - val_loss: 0.5461 - val_accuracy: 0.6591
Epoch 938/1000
101/101 [==============================] - 0s 159us/step - loss: 0.4733 - accuracy: 0.7525 - val_loss: 0.5456 - val_accuracy: 0.6591
Epoch 939/1000
101/101 [==============================] - 0s 152us/step - loss: 0.4727 - accuracy: 0.7525 - val_loss: 0.5371 - val_accuracy: 0.6591
Epoch 940/1000
101/101 [==============================] - 0s 132us/step - loss: 0.4720 - accuracy: 0.7525 - val_loss: 0.5364 - val_accuracy: 0.6591
Epoch 941/1000
101/101 [==============================] - 0s 156us/step - loss: 0.4718 - accuracy: 0.7525 - val_loss: 0.5361 - val_accuracy: 0.6591
Epoch 942/1000
101/101 [==============================] - 0s 125us/step - loss: 0.4718 - accuracy: 0.7525 - val_loss: 0.5457 - 

101/101 [==============================] - 0s 151us/step - loss: 0.4686 - accuracy: 0.7525 - val_loss: 0.5436 - val_accuracy: 0.6591
Epoch 992/1000
101/101 [==============================] - 0s 153us/step - loss: 0.4693 - accuracy: 0.7525 - val_loss: 0.5489 - val_accuracy: 0.6591
Epoch 993/1000
101/101 [==============================] - 0s 156us/step - loss: 0.4693 - accuracy: 0.7525 - val_loss: 0.5449 - val_accuracy: 0.6591
Epoch 994/1000
101/101 [==============================] - 0s 126us/step - loss: 0.4682 - accuracy: 0.7525 - val_loss: 0.5314 - val_accuracy: 0.6591
Epoch 995/1000
101/101 [==============================] - 0s 151us/step - loss: 0.4679 - accuracy: 0.7525 - val_loss: 0.5347 - val_accuracy: 0.6591
Epoch 996/1000
101/101 [==============================] - 0s 131us/step - loss: 0.4679 - accuracy: 0.7525 - val_loss: 0.5472 - val_accuracy: 0.6591
Epoch 997/1000
101/101 [==============================] - 0s 144us/step - loss: 0.4675 - accuracy: 0.7525 - val_loss: 0.5580 - 

In [58]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

44/44 [==============================] - 0s 118us/step
combination test accuracy: 65.91%


In [59]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [60]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [61]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/1000
242/242 [==============================] - 0s 417us/step - loss: 15.8967 - accuracy: 0.4711 - val_loss: 0.7020 - val_accuracy: 0.4615
Epoch 2/1000
242/242 [==============================] - 0s 81us/step - loss: 0.7071 - accuracy: 0.4587 - val_loss: 0.7107 - val_accuracy: 0.4615
Epoch 3/1000
242/242 [==============================] - 0s 118us/step - loss: 0.7074 - accuracy: 0.4504 - val_loss: 0.6974 - val_accuracy: 0.4519
Epoch 4/1000
242/242 [==============================] - 0s 109us/step - loss: 0.7010 - accuracy: 0.4711 - val_loss: 0.6924 - val_accuracy: 0.5288
Epoch 5/1000
242/242 [==============================] - 0s 116us/step - loss: 0.7010 - accuracy: 0.4380 - val_loss: 0.6961 - val_accuracy: 0.4327
Epoch 6/1000
242/242 [==============================] - 0s 116us/step - loss: 0.6975 - accuracy: 0.4545 - val_loss: 0.6930 - val_accuracy: 0.4712
Epoch 7/1000
242/242 [==============================] - 0s 124us/step - loss: 

242/242 [==============================] - 0s 136us/step - loss: 0.6690 - accuracy: 0.6529 - val_loss: 0.6889 - val_accuracy: 0.5288
Epoch 57/1000
242/242 [==============================] - 0s 136us/step - loss: 0.6701 - accuracy: 0.6281 - val_loss: 0.6826 - val_accuracy: 0.5769
Epoch 58/1000
242/242 [==============================] - 0s 173us/step - loss: 0.6699 - accuracy: 0.6322 - val_loss: 0.6854 - val_accuracy: 0.5288
Epoch 59/1000
242/242 [==============================] - 0s 197us/step - loss: 0.6703 - accuracy: 0.6116 - val_loss: 0.6951 - val_accuracy: 0.5192
Epoch 60/1000
242/242 [==============================] - 0s 152us/step - loss: 0.6673 - accuracy: 0.6240 - val_loss: 0.6839 - val_accuracy: 0.5288
Epoch 61/1000
242/242 [==============================] - 0s 115us/step - loss: 0.6662 - accuracy: 0.6322 - val_loss: 0.6835 - val_accuracy: 0.5288
Epoch 62/1000
242/242 [==============================] - 0s 154us/step - loss: 0.6654 - accuracy: 0.6281 - val_loss: 0.6872 - val_ac

Epoch 112/1000
242/242 [==============================] - 0s 111us/step - loss: 0.6553 - accuracy: 0.6281 - val_loss: 0.6712 - val_accuracy: 0.5385
Epoch 113/1000
242/242 [==============================] - 0s 117us/step - loss: 0.6483 - accuracy: 0.6281 - val_loss: 0.6757 - val_accuracy: 0.5192
Epoch 114/1000
242/242 [==============================] - 0s 124us/step - loss: 0.6485 - accuracy: 0.6240 - val_loss: 0.6686 - val_accuracy: 0.5769
Epoch 115/1000
242/242 [==============================] - 0s 91us/step - loss: 0.6494 - accuracy: 0.6364 - val_loss: 0.6696 - val_accuracy: 0.5673
Epoch 116/1000
242/242 [==============================] - 0s 87us/step - loss: 0.6528 - accuracy: 0.6488 - val_loss: 0.6688 - val_accuracy: 0.5673
Epoch 117/1000
242/242 [==============================] - 0s 93us/step - loss: 0.6646 - accuracy: 0.6240 - val_loss: 0.6767 - val_accuracy: 0.5385
Epoch 118/1000
242/242 [==============================] - 0s 88us/step - loss: 0.6568 - accuracy: 0.6281 - val_loss

Epoch 168/1000
242/242 [==============================] - 0s 87us/step - loss: 0.6293 - accuracy: 0.6694 - val_loss: 0.6684 - val_accuracy: 0.5385
Epoch 169/1000
242/242 [==============================] - 0s 92us/step - loss: 0.6575 - accuracy: 0.6281 - val_loss: 0.6552 - val_accuracy: 0.5769
Epoch 170/1000
242/242 [==============================] - 0s 91us/step - loss: 0.6285 - accuracy: 0.6818 - val_loss: 0.6738 - val_accuracy: 0.5385
Epoch 171/1000
242/242 [==============================] - 0s 87us/step - loss: 0.6380 - accuracy: 0.6529 - val_loss: 0.6613 - val_accuracy: 0.5673
Epoch 172/1000
242/242 [==============================] - 0s 90us/step - loss: 0.6509 - accuracy: 0.6364 - val_loss: 0.6840 - val_accuracy: 0.5288
Epoch 173/1000
242/242 [==============================] - 0s 76us/step - loss: 0.6316 - accuracy: 0.6653 - val_loss: 0.6697 - val_accuracy: 0.5385
Epoch 174/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6273 - accuracy: 0.6694 - val_loss: 0

Epoch 224/1000
242/242 [==============================] - 0s 82us/step - loss: 0.6275 - accuracy: 0.6446 - val_loss: 0.7239 - val_accuracy: 0.5288
Epoch 225/1000
242/242 [==============================] - 0s 83us/step - loss: 0.6621 - accuracy: 0.6405 - val_loss: 0.6522 - val_accuracy: 0.5385
Epoch 226/1000
242/242 [==============================] - 0s 86us/step - loss: 0.6200 - accuracy: 0.6488 - val_loss: 0.6482 - val_accuracy: 0.5673
Epoch 227/1000
242/242 [==============================] - 0s 87us/step - loss: 0.6263 - accuracy: 0.6570 - val_loss: 0.6462 - val_accuracy: 0.5865
Epoch 228/1000
242/242 [==============================] - 0s 79us/step - loss: 0.6255 - accuracy: 0.6488 - val_loss: 0.6433 - val_accuracy: 0.5865
Epoch 229/1000
242/242 [==============================] - 0s 88us/step - loss: 0.6315 - accuracy: 0.6736 - val_loss: 0.6429 - val_accuracy: 0.6058
Epoch 230/1000
242/242 [==============================] - 0s 72us/step - loss: 0.6284 - accuracy: 0.6446 - val_loss: 0

...

242/242 [==============================] - 0s 103us/step - loss: 0.5799 - accuracy: 0.6942 - val_loss: 0.6303 - val_accuracy: 0.5865
Epoch 728/1000
242/242 [==============================] - 0s 110us/step - loss: 0.5769 - accuracy: 0.6901 - val_loss: 0.6541 - val_accuracy: 0.5385
Epoch 729/1000
242/242 [==============================] - 0s 117us/step - loss: 0.5699 - accuracy: 0.6942 - val_loss: 0.6269 - val_accuracy: 0.5962
Epoch 730/1000
242/242 [==============================] - 0s 120us/step - loss: 0.5549 - accuracy: 0.7107 - val_loss: 0.6453 - val_accuracy: 0.5288
Epoch 731/1000
242/242 [==============================] - 0s 92us/step - loss: 0.5624 - accuracy: 0.6860 - val_loss: 0.6573 - val_accuracy: 0.5288
Epoch 732/1000
242/242 [==============================] - 0s 89us/step - loss: 0.6033 - accuracy: 0.6736 - val_loss: 0.6468 - val_accuracy: 0.5385
Epoch 733/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5667 - accuracy: 0.6860 - val_loss: 0.6236 - val

Epoch 783/1000
242/242 [==============================] - 0s 74us/step - loss: 0.5640 - accuracy: 0.6942 - val_loss: 0.6343 - val_accuracy: 0.5865
Epoch 784/1000
242/242 [==============================] - 0s 67us/step - loss: 0.5720 - accuracy: 0.7066 - val_loss: 0.6237 - val_accuracy: 0.5962
Epoch 785/1000
242/242 [==============================] - 0s 61us/step - loss: 0.5578 - accuracy: 0.6983 - val_loss: 0.6298 - val_accuracy: 0.5962
Epoch 786/1000
242/242 [==============================] - 0s 61us/step - loss: 0.5797 - accuracy: 0.6818 - val_loss: 0.6246 - val_accuracy: 0.5962
Epoch 787/1000
242/242 [==============================] - 0s 65us/step - loss: 0.5572 - accuracy: 0.7149 - val_loss: 0.6238 - val_accuracy: 0.5962
Epoch 788/1000
242/242 [==============================] - 0s 59us/step - loss: 0.5717 - accuracy: 0.6901 - val_loss: 0.6571 - val_accuracy: 0.5385
Epoch 789/1000
242/242 [==============================] - 0s 59us/step - loss: 0.5561 - accuracy: 0.7025 - val_loss: 0

242/242 [==============================] - 0s 102us/step - loss: 0.5741 - accuracy: 0.6901 - val_loss: 0.6624 - val_accuracy: 0.5385
Epoch 839/1000
242/242 [==============================] - 0s 114us/step - loss: 0.5832 - accuracy: 0.6777 - val_loss: 0.6317 - val_accuracy: 0.5962
Epoch 840/1000
242/242 [==============================] - 0s 113us/step - loss: 0.5621 - accuracy: 0.7066 - val_loss: 0.6225 - val_accuracy: 0.5962
Epoch 841/1000
242/242 [==============================] - 0s 108us/step - loss: 0.6054 - accuracy: 0.6570 - val_loss: 0.6234 - val_accuracy: 0.5962
Epoch 842/1000
242/242 [==============================] - 0s 118us/step - loss: 0.5648 - accuracy: 0.6942 - val_loss: 0.7190 - val_accuracy: 0.5288
Epoch 843/1000
242/242 [==============================] - 0s 127us/step - loss: 0.5966 - accuracy: 0.6818 - val_loss: 0.6267 - val_accuracy: 0.5962
Epoch 844/1000
242/242 [==============================] - 0s 120us/step - loss: 0.5503 - accuracy: 0.7066 - val_loss: 0.6354 - 

Epoch 894/1000
242/242 [==============================] - 0s 77us/step - loss: 0.5686 - accuracy: 0.6983 - val_loss: 0.6234 - val_accuracy: 0.5962
Epoch 895/1000
242/242 [==============================] - 0s 73us/step - loss: 0.5580 - accuracy: 0.6901 - val_loss: 0.6219 - val_accuracy: 0.5962
Epoch 896/1000
242/242 [==============================] - 0s 63us/step - loss: 0.5578 - accuracy: 0.6942 - val_loss: 0.6251 - val_accuracy: 0.5962
Epoch 897/1000
242/242 [==============================] - 0s 71us/step - loss: 0.5505 - accuracy: 0.7066 - val_loss: 0.6230 - val_accuracy: 0.6058
Epoch 898/1000
242/242 [==============================] - 0s 73us/step - loss: 0.5514 - accuracy: 0.7066 - val_loss: 0.6250 - val_accuracy: 0.5962
Epoch 899/1000
242/242 [==============================] - 0s 63us/step - loss: 0.5540 - accuracy: 0.7025 - val_loss: 0.6256 - val_accuracy: 0.5962
Epoch 900/1000
242/242 [==============================] - 0s 65us/step - loss: 0.5679 - accuracy: 0.6983 - val_loss: 0

Epoch 950/1000
242/242 [==============================] - 0s 86us/step - loss: 0.5638 - accuracy: 0.6860 - val_loss: 0.6326 - val_accuracy: 0.5769
Epoch 951/1000
242/242 [==============================] - 0s 71us/step - loss: 0.5578 - accuracy: 0.6901 - val_loss: 0.6236 - val_accuracy: 0.5962
Epoch 952/1000
242/242 [==============================] - 0s 73us/step - loss: 0.5937 - accuracy: 0.6983 - val_loss: 0.6621 - val_accuracy: 0.5385
Epoch 953/1000
242/242 [==============================] - 0s 75us/step - loss: 0.5806 - accuracy: 0.6942 - val_loss: 0.7163 - val_accuracy: 0.5385
Epoch 954/1000
242/242 [==============================] - 0s 82us/step - loss: 0.5677 - accuracy: 0.7025 - val_loss: 0.6224 - val_accuracy: 0.5865
Epoch 955/1000
242/242 [==============================] - 0s 80us/step - loss: 0.5643 - accuracy: 0.7025 - val_loss: 0.6345 - val_accuracy: 0.5865
Epoch 956/1000
242/242 [==============================] - 0s 76us/step - loss: 0.5457 - accuracy: 0.7025 - val_loss: 0

In [173]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

147/147 [==============================] - 0s 112us/step
combination test accuracy: 82.31%


In [ ]:
############ Logistic Regression ############

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [63]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 88.64%


In [65]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 76.92%


In [67]:
############## Random Forest ##############

In [68]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [69]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 97.73%


In [70]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [71]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 81.73%


In [72]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [73]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.95238095 0.95238095 0.95       1.         0.89473684]
0.9498997493734335


In [74]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.76       0.83333333 0.625      0.70833333 0.77083333]
0.7395
